In [3]:
# RAG System Evaluation Notebook

import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import List, Tuple
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from bert_score import score as bert_score
import matplotlib.pyplot as plt
from RAG_UTILS import RAGSystem, DocumentProcessor  # Import your existing RAG system

# Constants
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
MODEL_ID = "nvidia/Mistral-NeMo-Minitron-8B-Base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
PDF_FOLDER_PATH = "local_database"  # Update this to your local database path

# Initialize RAG system
rag_system = RAGSystem(
    embedding_model_name=EMBEDDING_MODEL_NAME,
    model_id=MODEL_ID,
    reranker_model=RERANKER_MODEL,
)

# Load and process documents
doc_processor = DocumentProcessor(EMBEDDING_MODEL_NAME)
raw_documents = doc_processor.load_pdfs_from_folder(PDF_FOLDER_PATH)
processed_documents = doc_processor.split_documents(raw_documents)

# Build vector database
knowledge_index = rag_system.build_vector_database(processed_documents)

# Evaluation function
def evaluate_rag(questions: List[str], reference_answers: List[str]) -> dict:
    results = []
    for question, reference in tqdm(zip(questions, reference_answers), total=len(questions)):
        answer, relevant_docs, _ = rag_system.answer_with_rag(question, knowledge_index)
        results.append({
            "question": question,
            "reference_answer": reference,
            "generated_answer": answer,
            "relevant_docs": relevant_docs
        })
    return results

# Load your evaluation dataset
# You should replace this with your actual evaluation dataset
eval_data = pd.read_csv("your_evaluation_dataset.csv")
questions = eval_data["question"].tolist()
reference_answers = eval_data["reference_answer"].tolist()

# Run evaluation
evaluation_results = evaluate_rag(questions, reference_answers)

# Helper functions for different metrics

def calculate_retrieval_metrics(results: List[dict]) -> dict:
    precisions, recalls, f1s = [], [], []
    for result in results:
        relevant_docs = set(result["relevant_docs"])
        retrieved_docs = set(result["generated_answer"].split())  # Simplification, adjust as needed
        precision = len(relevant_docs.intersection(retrieved_docs)) / len(retrieved_docs) if retrieved_docs else 0
        recall = len(relevant_docs.intersection(retrieved_docs)) / len(relevant_docs) if relevant_docs else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    return {
        "avg_precision": np.mean(precisions),
        "avg_recall": np.mean(recalls),
        "avg_f1": np.mean(f1s)
    }

def calculate_bertscore(results: List[dict]) -> float:
    references = [r["reference_answer"] for r in results]
    candidates = [r["generated_answer"] for r in results]
    P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
    return F1.mean().item()

def calculate_semantic_similarity(results: List[dict]) -> float:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    similarities = []
    for result in results:
        ref_embedding = model.encode(result["reference_answer"])
        gen_embedding = model.encode(result["generated_answer"])
        similarity = np.dot(ref_embedding, gen_embedding) / (REMOVED_SECRET(ref_embedding) * REMOVED_SECRET(gen_embedding))
        similarities.append(similarity)
    return np.mean(similarities)

# Calculate metrics
retrieval_metrics = calculate_retrieval_metrics(evaluation_results)
bertscore = calculate_bertscore(evaluation_results)
semantic_similarity = calculate_semantic_similarity(evaluation_results)

# Print results
print("Retrieval Metrics:")
print(f"Average Precision: {retrieval_metrics['avg_precision']:.4f}")
print(f"Average Recall: {retrieval_metrics['avg_recall']:.4f}")
print(f"Average F1 Score: {retrieval_metrics['avg_f1']:.4f}")
print(f"BERTScore: {bertscore:.4f}")
print(f"Semantic Similarity: {semantic_similarity:.4f}")

# Visualize results
metrics = ['Precision', 'Recall', 'F1 Score', 'BERTScore', 'Semantic Similarity']
values = [retrieval_metrics['avg_precision'], retrieval_metrics['avg_recall'], 
          retrieval_metrics['avg_f1'], bertscore, semantic_similarity]

plt.figure(figsize=(10, 6))
plt.bar(metrics, values)
plt.title('RAG System Evaluation Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
for i, v in enumerate(values):
    plt.text(i, v + 0.01, f'{v:.2f}', ha='center')
plt.show()

# Save results to CSV
results_df = pd.DataFrame(evaluation_results)
results_df.to_csv("rag_evaluation_results.csv", index=False)

print("Evaluation complete. Results saved to 'rag_evaluation_results.csv'")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `REMOVED_SECRET.GradScaler(args...)` is deprecated. Please use `REMOVED_SECRET('cuda', args...)` instead.
  self.scaler = REMOVED_SECRET.GradScaler()
Loading PDFs: 100%|██████████| 30/30 [00:03<00:00,  9.45it/s]


Documents: [Document(page_content="Question:\n\nwho owns st andrews golf course in scotland\n\nAnswer:\n\n<P> The Old Course at St Andrews is considered the oldest golf course in the world , a public\n\ncourse over common land in St Andrews , Fife , Scotland . It is held in trust by The St Andrews\n\nLinks Trust under an act of Parliament . The Royal and Ancient Golf Club of St Andrews club house\n\nsits adjacent to the first tee , although it is but one of many clubs that have playing privileges on\n\nthe course , along with the general public . </P>\n\nContext:\n\nOld Course at St Andrews - wikipedia Old Course at St Andrews Jump to :\n\nnavigation , search Old Course R&A Clubhouse and 18th green in 2004 Club\n\ninformation\n\nCoordinates\n\n56 ° 20\n\n35 ''\n\nN 2 ° 48\n\n11 ''\n\n56.343 °\n\nN\n\n2.803\n\n°\n\n/ 56.343 ; - 2.803 Coordinates : 56 ° 20 35 '' N 2 ° 48 11 ''\n\nW\n\n56.343\n\n°\n\nN\n\n2.803\n\n°\n\n/ 56.343 ; - 2.803 Location St Andrews ,\n\nW\n\n/\n\nW\n\n/\n\nW\n\nS

Splitting Documents: 100%|██████████| 30/30 [00:00<00:00, 259.48it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'your_evaluation_dataset.csv'

In [1]:
# RAG System Evaluation Notebook for Local PDFs

import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import List, Tuple
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sentence_transformers import SentenceTransformer
from bert_score import score as bert_score
import matplotlib.pyplot as plt
from RAG_UTILS import RAGSystem, DocumentProcessor  # Import your existing RAG system
import random

# Constants
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
MODEL_ID = "nvidia/Mistral-NeMo-Minitron-8B-Base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
PDF_FOLDER_PATH = "local_database"  # Update this to your local database path
NUM_EVALUATION_SAMPLES = 50  # Number of questions to generate for evaluation

# Initialize RAG system
rag_system = RAGSystem(
    embedding_model_name=EMBEDDING_MODEL_NAME,
    model_id=MODEL_ID,
    reranker_model=RERANKER_MODEL,
)

# Load and process documents
doc_processor = DocumentProcessor(EMBEDDING_MODEL_NAME)
raw_documents = doc_processor.load_pdfs_from_folder(PDF_FOLDER_PATH)
processed_documents = doc_processor.split_documents(raw_documents)

# Build vector database
knowledge_index = rag_system.build_vector_database(processed_documents)

# Initialize question generator
question_generator = pipeline("text2text-generation", model="facebook/bart-large-cnn", device='cuda')

def generate_questions(documents: List[dict], num_questions: int) -> List[Tuple[str, str]]:
    questions_and_contexts = []
    for _ in range(num_questions):
        doc = random.choice(documents)
        context = doc.page_content
        question = question_generator(f"Generate a question based on this text: {context}", max_length=100, num_return_sequences=1)[0]['generated_text']
        questions_and_contexts.append((question, context))
    return questions_and_contexts

# Generate evaluation questions
eval_questions = generate_questions(processed_documents, NUM_EVALUATION_SAMPLES)

# Evaluation function
def evaluate_rag(questions_and_contexts: List[Tuple[str, str]]) -> List[dict]:
    results = []
    for question, context in tqdm(questions_and_contexts, total=len(questions_and_contexts)):
        answer, relevant_docs, _ = rag_system.answer_with_rag(question, knowledge_index)
        results.append({
            "question": question,
            "context": context,
            "generated_answer": answer,
            "relevant_docs": relevant_docs
        })
    return results

# Run evaluation
evaluation_results = evaluate_rag(eval_questions)

# Helper functions for different metrics

def calculate_retrieval_metrics(results: List[dict]) -> dict:
    precisions, recalls, f1s = [], [], []
    for result in results:
        relevant_docs = set([doc.split()[:10] for doc in result["relevant_docs"]])  # Use first 10 words as identifier
        retrieved_docs = set([doc.split()[:10] for doc in result["generated_answer"].split('\n')])  # Simplification, adjust as needed
        precision = len(relevant_docs.intersection(retrieved_docs)) / len(retrieved_docs) if retrieved_docs else 0
        recall = len(relevant_docs.intersection(retrieved_docs)) / len(relevant_docs) if relevant_docs else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    return {
        "avg_precision": np.mean(precisions),
        "avg_recall": np.mean(recalls),
        "avg_f1": np.mean(f1s)
    }

def calculate_bertscore(results: List[dict]) -> float:
    references = [r["context"] for r in results]
    candidates = [r["generated_answer"] for r in results]
    P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
    return F1.mean().item()

def calculate_semantic_similarity(results: List[dict]) -> float:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    similarities = []
    for result in results:
        context_embedding = model.encode(result["context"])
        gen_embedding = model.encode(result["generated_answer"])
        similarity = np.dot(context_embedding, gen_embedding) / (REMOVED_SECRET(context_embedding) * REMOVED_SECRET(gen_embedding))
        similarities.append(similarity)
    return np.mean(similarities)

# Calculate metrics
retrieval_metrics = calculate_retrieval_metrics(evaluation_results)
bertscore = calculate_bertscore(evaluation_results)
semantic_similarity = calculate_semantic_similarity(evaluation_results)

# Print results
print("Retrieval Metrics:")
print(f"Average Precision: {retrieval_metrics['avg_precision']:.4f}")
print(f"Average Recall: {retrieval_metrics['avg_recall']:.4f}")
print(f"Average F1 Score: {retrieval_metrics['avg_f1']:.4f}")
print(f"BERTScore: {bertscore:.4f}")
print(f"Semantic Similarity: {semantic_similarity:.4f}")

# Visualize results
metrics = ['Precision', 'Recall', 'F1 Score', 'BERTScore', 'Semantic Similarity']
values = [retrieval_metrics['avg_precision'], retrieval_metrics['avg_recall'], 
          retrieval_metrics['avg_f1'], bertscore, semantic_similarity]

plt.figure(figsize=(10, 6))
plt.bar(metrics, values)
plt.title('RAG System Evaluation Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
for i, v in enumerate(values):
    plt.text(i, v + 0.01, f'{v:.2f}', ha='center')
plt.show()

# Save results to CSV
results_df = pd.DataFrame(evaluation_results)
results_df.to_csv("rag_evaluation_results.csv", index=False)

print("Evaluation complete. Results saved to 'rag_evaluation_results.csv'")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `REMOVED_SECRET.GradScaler(args...)` is deprecated. Please use `REMOVED_SECRET('cuda', args...)` instead.
  self.scaler = REMOVED_SECRET.GradScaler()
Loading PDFs: 100%|██████████| 30/30 [00:03<00:00,  9.88it/s]


Documents: [Document(page_content="Question:\n\nwho owns st andrews golf course in scotland\n\nAnswer:\n\n<P> The Old Course at St Andrews is considered the oldest golf course in the world , a public\n\ncourse over common land in St Andrews , Fife , Scotland . It is held in trust by The St Andrews\n\nLinks Trust under an act of Parliament . The Royal and Ancient Golf Club of St Andrews club house\n\nsits adjacent to the first tee , although it is but one of many clubs that have playing privileges on\n\nthe course , along with the general public . </P>\n\nContext:\n\nOld Course at St Andrews - wikipedia Old Course at St Andrews Jump to :\n\nnavigation , search Old Course R&A Clubhouse and 18th green in 2004 Club\n\ninformation\n\nCoordinates\n\n56 ° 20\n\n35 ''\n\nN 2 ° 48\n\n11 ''\n\n56.343 °\n\nN\n\n2.803\n\n°\n\n/ 56.343 ; - 2.803 Coordinates : 56 ° 20 35 '' N 2 ° 48 11 ''\n\nW\n\n56.343\n\n°\n\nN\n\n2.803\n\n°\n\n/ 56.343 ; - 2.803 Location St Andrews ,\n\nW\n\n/\n\nW\n\n/\n\nW\n\nS

Splitting Documents: 100%|██████████| 30/30 [00:00<00:00, 272.07it/s]
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  0%|          | 0/50 [00:00<?, ?it/s]

=> Retrieving documents...
Retrieved 30 documents.
=> Reranking documents...


...

=> Retrieving documents...
Retrieved 30 documents.
=> Reranking documents...


100%|██████████| 1/1 [00:08<00:00,  8.06s/it]


Reranked to 1 documents.
Using 1 documents for answering.
=> Generating answer...


100%|██████████| 50/50 [1:44:21<00:00, 125.23s/it]


TypeError: unhashable type: 'list'

In [3]:
# Complete RAG System Evaluation Notebook for Local PDFs

import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import List, Tuple
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sentence_transformers import SentenceTransformer
from bert_score import score as bert_score
import matplotlib.pyplot as plt
from RAG_UTILS import RAGSystem, DocumentProcessor  # Import your existing RAG system
import random

# Constants
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
MODEL_ID = "nvidia/Mistral-NeMo-Minitron-8B-Base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
PDF_FOLDER_PATH = "local_database"  # Update this to your local database path
NUM_EVALUATION_SAMPLES = 50  # Number of questions to generate for evaluation

# Initialize RAG system
rag_system = RAGSystem(
    embedding_model_name=EMBEDDING_MODEL_NAME,
    model_id=MODEL_ID,
    reranker_model=RERANKER_MODEL,
)

# Function to convert PDFs to CSV
def convert_pdfs_to_csv(pdf_folder_path: str, output_csv_path: str):
    doc_processor = DocumentProcessor(EMBEDDING_MODEL_NAME)
    raw_documents = doc_processor.load_pdfs_from_folder(pdf_folder_path)
    processed_documents = doc_processor.split_documents(raw_documents)
    
    data = []
    for doc in processed_documents:
        data.append({
            "content": doc.page_content,
            "source": REMOVED_SECRET("source", ""),
            "page": REMOVED_SECRET("page", "")
        })
    
    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)
    print(f"CSV file saved to {output_csv_path}")

# Convert PDFs to CSV (run this once)
convert_pdfs_to_csv(PDF_FOLDER_PATH, "processed_pdfs.csv")

# Function to load data from CSV
def load_from_csv(csv_path: str) -> List[dict]:
    df = pd.read_csv(csv_path)
    return df.to_dict('records')

# Load processed documents from CSV
processed_documents = load_from_csv("processed_pdfs.csv")

# Build vector database
knowledge_index = rag_system.build_vector_database(processed_documents)

# Initialize question generator
question_generator = pipeline("text2text-generation", model="facebook/bart-large-cnn", device='cuda')

def generate_questions(documents: List[dict], num_questions: int) -> List[Tuple[str, str]]:
    questions_and_contexts = []
    for _ in range(num_questions):
        doc = random.choice(documents)
        context = doc['content']
        question = question_generator(f"Generate a question based on this text: {context}", max_length=50, num_return_sequences=1)[0]['generated_text']
        questions_and_contexts.append((question, context))
    return questions_and_contexts

# Generate evaluation questions
eval_questions = generate_questions(processed_documents, NUM_EVALUATION_SAMPLES)

# Evaluation function
def evaluate_rag(questions_and_contexts: List[Tuple[str, str]]) -> List[dict]:
    results = []
    for question, context in tqdm(questions_and_contexts, total=len(questions_and_contexts)):
        answer, relevant_docs, _ = rag_system.answer_with_rag(question, knowledge_index)
        results.append({
            "question": question,
            "context": context,
            "generated_answer": answer,
            "relevant_docs": relevant_docs
        })
    return results

# Run evaluation
evaluation_results = evaluate_rag(eval_questions)

# Helper functions for different metrics

def calculate_retrieval_metrics(results: List[dict]) -> dict:
    precisions, recalls, f1s = [], [], []
    for result in results:
        relevant_docs = set(tuple(doc.split()[:10]) for doc in result["relevant_docs"])  # Use first 10 words as identifier
        retrieved_docs = set(tuple(doc.split()[:10]) for doc in result["generated_answer"].split('\n'))  # Simplification, adjust as needed
        precision = len(relevant_docs.intersection(retrieved_docs)) / len(retrieved_docs) if retrieved_docs else 0
        recall = len(relevant_docs.intersection(retrieved_docs)) / len(relevant_docs) if relevant_docs else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    return {
        "avg_precision": np.mean(precisions),
        "avg_recall": np.mean(recalls),
        "avg_f1": np.mean(f1s)
    }

def calculate_bertscore(results: List[dict]) -> float:
    references = [r["context"] for r in results]
    candidates = [r["generated_answer"] for r in results]
    P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
    return F1.mean().item()

def calculate_semantic_similarity(results: List[dict]) -> float:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    similarities = []
    for result in results:
        context_embedding = model.encode(result["context"])
        gen_embedding = model.encode(result["generated_answer"])
        similarity = np.dot(context_embedding, gen_embedding) / (REMOVED_SECRET(context_embedding) * REMOVED_SECRET(gen_embedding))
        similarities.append(similarity)
    return np.mean(similarities)

# Calculate metrics
retrieval_metrics = calculate_retrieval_metrics(evaluation_results)
bertscore = calculate_bertscore(evaluation_results)
semantic_similarity = calculate_semantic_similarity(evaluation_results)

# Print results
print("Retrieval Metrics:")
print(f"Average Precision: {retrieval_metrics['avg_precision']:.4f}")
print(f"Average Recall: {retrieval_metrics['avg_recall']:.4f}")
print(f"Average F1 Score: {retrieval_metrics['avg_f1']:.4f}")
print(f"BERTScore: {bertscore:.4f}")
print(f"Semantic Similarity: {semantic_similarity:.4f}")

# Visualize results
metrics = ['Precision', 'Recall', 'F1 Score', 'BERTScore', 'Semantic Similarity']
values = [retrieval_metrics['avg_precision'], retrieval_metrics['avg_recall'], 
          retrieval_metrics['avg_f1'], bertscore, semantic_similarity]

plt.figure(figsize=(10, 6))
plt.bar(metrics, values)
plt.title('RAG System Evaluation Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
for i, v in enumerate(values):
    plt.text(i, v + 0.01, f'{v:.2f}', ha='center')
plt.show()

# Save results to CSV
results_df = pd.DataFrame(evaluation_results)
results_df.to_csv("rag_evaluation_results.csv", index=False)

print("Evaluation complete. Results saved to 'rag_evaluation_results.csv'")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `REMOVED_SECRET.GradScaler(args...)` is deprecated. Please use `REMOVED_SECRET('cuda', args...)` instead.
  self.scaler = REMOVED_SECRET.GradScaler()
Loading PDFs: 100%|██████████| 30/30 [00:04<00:00,  7.32it/s]


Documents: [Document(page_content="Question:\n\nwho owns st andrews golf course in scotland\n\nAnswer:\n\n<P> The Old Course at St Andrews is considered the oldest golf course in the world , a public\n\ncourse over common land in St Andrews , Fife , Scotland . It is held in trust by The St Andrews\n\nLinks Trust under an act of Parliament . The Royal and Ancient Golf Club of St Andrews club house\n\nsits adjacent to the first tee , although it is but one of many clubs that have playing privileges on\n\nthe course , along with the general public . </P>\n\nContext:\n\nOld Course at St Andrews - wikipedia Old Course at St Andrews Jump to :\n\nnavigation , search Old Course R&A Clubhouse and 18th green in 2004 Club\n\ninformation\n\nCoordinates\n\n56 ° 20\n\n35 ''\n\nN 2 ° 48\n\n11 ''\n\n56.343 °\n\nN\n\n2.803\n\n°\n\n/ 56.343 ; - 2.803 Coordinates : 56 ° 20 35 '' N 2 ° 48 11 ''\n\nW\n\n56.343\n\n°\n\nN\n\n2.803\n\n°\n\n/ 56.343 ; - 2.803 Location St Andrews ,\n\nW\n\n/\n\nW\n\n/\n\nW\n\nS

Splitting Documents: 100%|██████████| 30/30 [00:00<00:00, 174.92it/s]


CSV file saved to processed_pdfs.csv


AttributeError: 'dict' object has no attribute 'page_content'

In [1]:
# Optimized RAG System Evaluation Notebook with Dataset

import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import List, Tuple, Dict
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sentence_transformers import SentenceTransformer
from bert_score import score as bert_score
import matplotlib.pyplot as plt
from RAG_UTILS import RAGSystem, DocumentProcessor, LangchainDocument  # Import your existing RAG system
from REMOVED_SECRET import Dataset, DataLoader
import random

# Constants
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
MODEL_ID = "nvidia/Mistral-NeMo-Minitron-8B-Base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
PDF_FOLDER_PATH = "local_database"  # Update this to your local database path
CSV_PATH = "processed_pdfs.csv"
NUM_EVALUATION_SAMPLES = 50  # Number of questions to generate for evaluation
BATCH_SIZE = 8  # Adjust based on your GPU memory

# Custom Dataset
class DocumentDataset(Dataset):
    def __init__(self, documents):
        self.documents = documents

    def __len__(self):
        return len(self.documents)

    def __getitem__(self, idx):
        return self.documents[idx]
    

def custom_collate_fn(batch):
    return batch

# Initialize RAG system
rag_system = RAGSystem(
    embedding_model_name=EMBEDDING_MODEL_NAME,
    model_id=MODEL_ID,
    reranker_model=RERANKER_MODEL,
)

# Function to load data from CSV
def load_from_csv(csv_path: str) -> List[LangchainDocument]:
    df = pd.read_csv(csv_path)
    return [
        LangchainDocument(page_content=row['content'], metadata={"source": row['source'], "page": row['page']})
        for _, row in df.iterrows()
    ]

# Load processed documents from CSV
processed_documents = load_from_csv(CSV_PATH)

# Create dataset and dataloader
document_dataset = DocumentDataset(processed_documents)
document_dataloader = DataLoader(document_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn)
# Build vector database
print("Building vector database...")
knowledge_index = rag_system.build_vector_database(processed_documents)

# Initialize question generator
question_generator = pipeline("text2text-generation", model="facebook/bart-large-cnn", device=0 if REMOVED_SECRET() else -1)

def generate_questions(dataloader: DataLoader, num_questions: int) -> List[Tuple[str, str]]:
    questions_and_contexts = []
    for batch in tqdm(dataloader, desc="Generating questions", total=min(len(dataloader), num_questions // BATCH_SIZE + 1)):
        contexts = [doc.page_content for doc in batch]
        questions = question_generator(
            [f"Generate a question based on this text: {context}" for context in contexts],
            max_length=50,
            num_return_sequences=1
        )
        questions_and_contexts.extend(list(zip([q[0]['generated_text'] for q in questions], contexts)))
        if len(questions_and_contexts) >= num_questions:
            break
    return questions_and_contexts[:num_questions]

# Generate evaluation questions
print("Generating evaluation questions...")
eval_questions = generate_questions(document_dataloader, NUM_EVALUATION_SAMPLES)

# Evaluation function
def evaluate_rag(questions_and_contexts: List[Tuple[str, str]]) -> List[dict]:
    results = []
    for question, context in tqdm(questions_and_contexts, desc="Evaluating RAG system"):
        answer, relevant_docs, _ = rag_system.answer_with_rag(question, knowledge_index)
        results.append({
            "question": question,
            "context": context,
            "generated_answer": answer,
            "relevant_docs": relevant_docs
        })
    return results

# Run evaluation
print("Running RAG system evaluation...")
evaluation_results = evaluate_rag(eval_questions)

def calculate_retrieval_metrics(results: List[dict]) -> dict:
    precisions, recalls, f1s = [], [], []
    for result in results:
        relevant_docs = set(tuple(doc.split()[:10]) for doc in result["relevant_docs"])  # Use first 10 words as identifier
        retrieved_docs = set(tuple(doc.split()[:10]) for doc in result["generated_answer"].split('\n'))  # Simplification, adjust as needed
        precision = len(relevant_docs.intersection(retrieved_docs)) / len(retrieved_docs) if retrieved_docs else 0
        recall = len(relevant_docs.intersection(retrieved_docs)) / len(relevant_docs) if relevant_docs else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    return {
        "avg_precision": np.mean(precisions),
        "avg_recall": np.mean(recalls),
        "avg_f1": np.mean(f1s)
    }

def calculate_bertscore(results: List[dict]) -> float:
    references = [r["context"] for r in results]
    candidates = [r["generated_answer"] for r in results]
    P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
    return F1.mean().item()

def calculate_semantic_similarity(results: List[dict]) -> float:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    similarities = []
    for result in results:
        context_embedding = model.encode(result["context"])
        gen_embedding = model.encode(result["generated_answer"])
        similarity = np.dot(context_embedding, gen_embedding) / (REMOVED_SECRET(context_embedding) * REMOVED_SECRET(gen_embedding))
        similarities.append(similarity)
    return np.mean(similarities)

# Calculate metrics
print("Calculating evaluation metrics...")
retrieval_metrics = calculate_retrieval_metrics(evaluation_results)
bertscore = calculate_bertscore(evaluation_results)
semantic_similarity = calculate_semantic_similarity(evaluation_results)

# Print results
print("\nRetrieval Metrics:")
print(f"Average Precision: {retrieval_metrics['avg_precision']:.4f}")
print(f"Average Recall: {retrieval_metrics['avg_recall']:.4f}")
print(f"Average F1 Score: {retrieval_metrics['avg_f1']:.4f}")
print(f"BERTScore: {bertscore:.4f}")
print(f"Semantic Similarity: {semantic_similarity:.4f}")

# Visualize results
metrics = ['Precision', 'Recall', 'F1 Score', 'BERTScore', 'Semantic Similarity']
values = [retrieval_metrics['avg_precision'], retrieval_metrics['avg_recall'], 
          retrieval_metrics['avg_f1'], bertscore, semantic_similarity]

plt.figure(figsize=(10, 6))
plt.bar(metrics, values)
plt.title('RAG System Evaluation Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
for i, v in enumerate(values):
    plt.text(i, v + 0.01, f'{v:.2f}', ha='center')
plt.show()

# Save results to CSV
results_df = pd.DataFrame(evaluation_results)
results_df.to_csv("rag_evaluation_results.csv", index=False)

print("Evaluation complete. Results saved to 'rag_evaluation_results.csv'")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `REMOVED_SECRET.GradScaler(args...)` is deprecated. Please use `REMOVED_SECRET('cuda', args...)` instead.
  self.scaler = REMOVED_SECRET.GradScaler()


Building vector database...


/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(


Generating evaluation questions...


Generating questions:   0%|          | 0/7 [00:00<?, ?it/s]/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1282: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
Generating questions:   0%|          | 0/7 [00:31<?, ?it/s]


KeyError: 0

In [1]:
# Optimized RAG System Evaluation Notebook with Corrected Question Generation

import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import List, Tuple, Dict
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sentence_transformers import SentenceTransformer
from bert_score import score as bert_score
import matplotlib.pyplot as plt
from RAG_UTILS import RAGSystem, DocumentProcessor, LangchainDocument  # Import your existing RAG system
from REMOVED_SECRET import Dataset, DataLoader
import random

# Constants
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
MODEL_ID = "nvidia/Mistral-NeMo-Minitron-8B-Base"
RERANKER_MODEL = None
PDF_FOLDER_PATH = "local_database"  # Update this to your local database path
CSV_PATH = "processed_pdfs.csv"
NUM_EVALUATION_SAMPLES = 50  # Number of questions to generate for evaluation
BATCH_SIZE = 8  # Adjust based on your GPU memory

# Custom Dataset
class DocumentDataset(Dataset):
    def __init__(self, documents):
        self.documents = documents

    def __len__(self):
        return len(self.documents)

    def __getitem__(self, idx):
        return self.documents[idx]

# Custom collate function
def custom_collate_fn(batch):
    return batch

# Initialize RAG system
rag_system = RAGSystem(
    embedding_model_name=EMBEDDING_MODEL_NAME,
    model_id=MODEL_ID,
    reranker_model=RERANKER_MODEL,
)

# Function to load data from CSV
def load_from_csv(csv_path: str) -> List[LangchainDocument]:
    df = pd.read_csv(csv_path)
    return [
        LangchainDocument(page_content=row['content'], metadata={"source": row['source'], "page": row['page']})
        for _, row in df.iterrows()
    ]

# Load processed documents from CSV
processed_documents = load_from_csv(CSV_PATH)

# Create dataset and dataloader
document_dataset = DocumentDataset(processed_documents)
document_dataloader = DataLoader(document_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn)

# Build vector database
print("Building vector database...")
knowledge_index = rag_system.build_vector_database(processed_documents)

# Initialize question generator
question_generator = pipeline("text2text-generation", model="facebook/bart-large-cnn", device=0 if REMOVED_SECRET() else -1)

def generate_questions(dataloader: DataLoader, num_questions: int) -> List[Tuple[str, str]]:
    questions_and_contexts = []
    for batch in tqdm(dataloader, desc="Generating questions", total=min(len(dataloader), num_questions // BATCH_SIZE + 1)):
        contexts = [doc.page_content for doc in batch]
        questions = question_generator(
            [f"Generate a question based on this text: {context[:500]}" for context in contexts],  # Limit context length
            max_length=50,
            num_return_sequences=1
        )
        questions_and_contexts.extend(list(zip([q['generated_text'] for q in questions], contexts)))
        if len(questions_and_contexts) >= num_questions:
            break
    return questions_and_contexts[:num_questions]

# Generate evaluation questions
print("Generating evaluation questions...")
eval_questions = generate_questions(document_dataloader, NUM_EVALUATION_SAMPLES)

# Evaluation function
def evaluate_rag(questions_and_contexts: List[Tuple[str, str]]) -> List[dict]:
    results = []
    for question, context in tqdm(questions_and_contexts, desc="Evaluating RAG system"):
        answer, relevant_docs, _ = rag_system.answer_with_rag(question, knowledge_index)
        results.append({
            "question": question,
            "context": context,
            "generated_answer": answer,
            "relevant_docs": relevant_docs
        })
    return results

# Run evaluation
print("Running RAG system evaluation...")
evaluation_results = evaluate_rag(eval_questions)

# Helper functions for different metrics
def calculate_retrieval_metrics(results: List[dict]) -> dict:
    precisions, recalls, f1s = [], [], []
    for result in results:
        relevant_docs = set(tuple(doc.split()[:10]) for doc in result["relevant_docs"])  # Use first 10 words as identifier
        retrieved_docs = set(tuple(doc.split()[:10]) for doc in result["generated_answer"].split('\n'))  # Simplification, adjust as needed
        precision = len(relevant_docs.intersection(retrieved_docs)) / len(retrieved_docs) if retrieved_docs else 0
        recall = len(relevant_docs.intersection(retrieved_docs)) / len(relevant_docs) if relevant_docs else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    return {
        "avg_precision": np.mean(precisions),
        "avg_recall": np.mean(recalls),
        "avg_f1": np.mean(f1s)
    }

def calculate_bertscore(results: List[dict]) -> float:
    references = [r["context"] for r in results]
    candidates = [r["generated_answer"] for r in results]
    P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
    return F1.mean().item()

def calculate_semantic_similarity(results: List[dict]) -> float:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    similarities = []
    for result in results:
        context_embedding = model.encode(result["context"])
        gen_embedding = model.encode(result["generated_answer"])
        similarity = np.dot(context_embedding, gen_embedding) / (REMOVED_SECRET(context_embedding) * REMOVED_SECRET(gen_embedding))
        similarities.append(similarity)
    return np.mean(similarities)

# Calculate metrics
print("Calculating evaluation metrics...")
retrieval_metrics = calculate_retrieval_metrics(evaluation_results)
bertscore = calculate_bertscore(evaluation_results)
semantic_similarity = calculate_semantic_similarity(evaluation_results)

# Print results
print("\nRetrieval Metrics:")
print(f"Average Precision: {retrieval_metrics['avg_precision']:.4f}")
print(f"Average Recall: {retrieval_metrics['avg_recall']:.4f}")
print(f"Average F1 Score: {retrieval_metrics['avg_f1']:.4f}")
print(f"BERTScore: {bertscore:.4f}")
print(f"Semantic Similarity: {semantic_similarity:.4f}")

# Visualize results
metrics = ['Precision', 'Recall', 'F1 Score', 'BERTScore', 'Semantic Similarity']
values = [retrieval_metrics['avg_precision'], retrieval_metrics['avg_recall'], 
          retrieval_metrics['avg_f1'], bertscore, semantic_similarity]

plt.figure(figsize=(10, 6))
plt.bar(metrics, values)
plt.title('RAG System Evaluation Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
for i, v in enumerate(values):
    plt.text(i, v + 0.01, f'{v:.2f}', ha='center')
plt.show()

# Save results to CSV
results_df = pd.DataFrame(evaluation_results)
results_df.to_csv("rag_evaluation_results.csv", index=False)

print("Evaluation complete. Results saved to 'rag_evaluation_results.csv'")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `REMOVED_SECRET.GradScaler(args...)` is deprecated. Please use `REMOVED_SECRET('cuda', args...)` instead.
  self.scaler = REMOVED_SECRET.GradScaler()


Building vector database...


/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(


Generating evaluation questions...


...

  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 665.04 seconds, 0.08 sentences/sec

Retrieval Metrics:
Average Precision: 0.0117
Average Recall: 0.0400
Average F1 Score: 0.0180
BERTScore: 0.5262
Semantic Similarity: 0.2411


Evaluation complete. Results saved to 'rag_evaluation_results.csv'


In [2]:
# Optimized RAG System Evaluation Notebook

import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import List, Tuple, Dict
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sentence_transformers import SentenceTransformer
from bert_score import score as bert_score
import matplotlib.pyplot as plt
from RAG_UTILS import RAGSystem, DocumentProcessor, LangchainDocument  # Import your existing RAG system
import random
import time

# Constants
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
MODEL_ID = "nvidia/Mistral-NeMo-Minitron-8B-Base"
RERANKER_MODEL = None  # Set to None to skip reranking
CSV_PATH = "processed_pdfs.csv"
NUM_EVALUATION_SAMPLES = 50
NUM_RETRIEVED_DOCS = 5  # Reduced from 30 to 5
MAX_NEW_TOKENS = 100  # Set maximum output length to 100 tokens

# Initialize RAG system
rag_system = RAGSystem(
    embedding_model_name=EMBEDDING_MODEL_NAME,
    model_id=MODEL_ID,
    reranker_model=RERANKER_MODEL,
)

# Load processed documents from CSV
def load_from_csv(csv_path: str) -> List[LangchainDocument]:
    df = pd.read_csv(csv_path)
    return [
        LangchainDocument(page_content=row['content'], metadata={"source": row['source'], "page": row['page']})
        for _, row in df.iterrows()
    ]

processed_documents = load_from_csv(CSV_PATH)

# Build vector database
print("Building vector database...")
start_time = time.time()
knowledge_index = rag_system.build_vector_database(processed_documents)
print(f"Vector database built in {time.time() - start_time:.2f} seconds")

# Initialize question generator
question_generator = pipeline("text2text-generation", model="facebook/bart-large-cnn", device=0 if REMOVED_SECRET() else -1)

def generate_questions(documents: List[LangchainDocument], num_questions: int) -> List[Tuple[str, str]]:
    questions_and_contexts = []
    for _ in tqdm(range(num_questions), desc="Generating questions"):
        doc = random.choice(documents)
        context = doc.page_content[:500]  # Limit context length
        question = question_generator(f"Generate a question based on this text: {context}", max_length=50, num_return_sequences=1)[0]['generated_text']
        questions_and_contexts.append((question, context))
    return questions_and_contexts

# Generate evaluation questions
print("Generating evaluation questions...")
start_time = time.time()
eval_questions = generate_questions(processed_documents, NUM_EVALUATION_SAMPLES)
print(f"Questions generated in {time.time() - start_time:.2f} seconds")

# Modified evaluation function
def evaluate_rag(questions_and_contexts: List[Tuple[str, str]]) -> List[dict]:
    results = []
    for question, context in tqdm(questions_and_contexts, desc="Evaluating RAG system"):
        # Retrieve documents
        start_time = time.time()
        relevant_docs = REMOVED_SECRET(query=question, k=NUM_RETRIEVED_DOCS)
        retrieve_time = time.time() - start_time
        
        # Generate answer
        start_time = time.time()
        answer, _, _ = rag_system.answer_with_rag(
            question=question,
            knowledge_index=knowledge_index,
            num_retrieved_docs=NUM_RETRIEVED_DOCS,
            num_docs_final=NUM_RETRIEVED_DOCS,  # No reranking, so use all retrieved docs
            max_new_tokens=MAX_NEW_TOKENS
        )
        generate_time = time.time() - start_time
        
        results.append({
            "question": question,
            "context": context,
            "generated_answer": answer,
            "relevant_docs": [doc.page_content for doc in relevant_docs],
            "retrieve_time": retrieve_time,
            "generate_time": generate_time
        })
    return results

# Run evaluation
print("Running RAG system evaluation...")
start_time = time.time()
evaluation_results = evaluate_rag(eval_questions)
total_eval_time = time.time() - start_time
print(f"Evaluation completed in {total_eval_time:.2f} seconds")

# Helper functions for different metrics
def calculate_retrieval_metrics(results: List[dict]) -> dict:
    precisions, recalls, f1s = [], [], []
    for result in results:
        relevant_set = set(result["context"].split())
        retrieved_set = set(result["generated_answer"].split())
        precision = len(relevant_set.intersection(retrieved_set)) / len(retrieved_set) if retrieved_set else 0
        recall = len(relevant_set.intersection(retrieved_set)) / len(relevant_set) if relevant_set else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    return {
        "avg_precision": np.mean(precisions),
        "avg_recall": np.mean(recalls),
        "avg_f1": np.mean(f1s)
    }

def calculate_semantic_similarity(results: List[dict]) -> float:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    similarities = []
    for result in results:
        context_embedding = model.encode(result["context"])
        gen_embedding = model.encode(result["generated_answer"])
        similarity = np.dot(context_embedding, gen_embedding) / (REMOVED_SECRET(context_embedding) * REMOVED_SECRET(gen_embedding))
        similarities.append(similarity)
    return np.mean(similarities)

# Calculate metrics
print("Calculating evaluation metrics...")
start_time = time.time()
retrieval_metrics = calculate_retrieval_metrics(evaluation_results)
semantic_similarity = calculate_semantic_similarity(evaluation_results)
print(f"Metrics calculated in {time.time() - start_time:.2f} seconds")

# Print results
print("\nRetrieval Metrics:")
print(f"Average Precision: {retrieval_metrics['avg_precision']:.4f}")
print(f"Average Recall: {retrieval_metrics['avg_recall']:.4f}")
print(f"Average F1 Score: {retrieval_metrics['avg_f1']:.4f}")
print(f"Semantic Similarity: {semantic_similarity:.4f}")

# Calculate and print timing statistics
retrieve_times = [r['retrieve_time'] for r in evaluation_results]
generate_times = [r['generate_time'] for r in evaluation_results]
print(f"\nAverage retrieval time: {np.mean(retrieve_times):.2f} seconds")
print(f"Average generation time: {np.mean(generate_times):.2f} seconds")
print(f"Total evaluation time: {total_eval_time:.2f} seconds")

# Visualize results
metrics = ['Precision', 'Recall', 'F1 Score', 'Semantic Similarity']
values = [retrieval_metrics['avg_precision'], retrieval_metrics['avg_recall'], 
          retrieval_metrics['avg_f1'], semantic_similarity]

plt.figure(figsize=(10, 6))
plt.bar(metrics, values)
plt.title('RAG System Evaluation Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
for i, v in enumerate(values):
    plt.text(i, v + 0.01, f'{v:.2f}', ha='center')
plt.show()

# Save results to CSV
results_df = pd.DataFrame(evaluation_results)
results_df.to_csv("rag_evaluation_results.csv", index=False)

print("Evaluation complete. Results saved to 'rag_evaluation_results.csv'")

/home/obb/codes/langers/.venv/lib/python3.10/site-packages/accelerate/utils/modeling.py:1405: UserWarning: Current model requires 5120 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Building vector database...
Vector database built in 6.42 seconds


/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(


Generating evaluation questions...


Generating questions:   0%|          | 0/50 [00:00<?, ?it/s]/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1282: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
Generating questions: 100%|██████████| 50/50 [13:57<00:00, 16.74s/it]


Questions generated in 837.18 seconds
Running RAG system evaluation...


Evaluating RAG system:   0%|          | 0/50 [00:00<?, ?it/s]


AttributeError: 'RAGSystem' object has no attribute 'knowledge_index'

In [7]:
# Updated RAG System Preprocessing Script

import os
import torch
import pandas as pd
from tqdm import tqdm
from typing import List, Tuple
from transformers import pipeline, AutoTokenizer
from REMOVED_SECRET import Dataset, DataLoader
from RAG_UTILS import DocumentProcessor, LangchainDocument
import nltk
from nltk.tokenize import sent_tokenize

# Download necessary NLTK data
# nltk.download('punkt')

# Constants
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
PDF_FOLDER_PATH = "local_database"
CSV_PATH = "processed_pdfs.csv"
QUESTIONS_PATH = "generated_questions.csv"
NUM_QUESTIONS = 1000  # Total number of questions to generate
BATCH_SIZE = 8

class DocumentDataset(Dataset):
    def __init__(self, documents):
        self.documents = documents

    def __len__(self):
        return len(self.documents)

    def __getitem__(self, idx):
        return self.documents[idx]

def custom_collate_fn(batch):
    return batch

def convert_pdfs_to_csv(pdf_folder_path: str, output_csv_path: str):
    doc_processor = DocumentProcessor(EMBEDDING_MODEL_NAME)
    raw_documents = doc_processor.load_pdfs_from_folder(pdf_folder_path)
    processed_documents = doc_processor.split_documents(raw_documents)
    
    data = []
    for doc in processed_documents:
        data.append({
            "content": doc.page_content,
            "source": REMOVED_SECRET("source", ""),
            "page": REMOVED_SECRET("page", "")
        })
    
    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)
    print(f"CSV file saved to {output_csv_path}")

def load_from_csv(csv_path: str) -> List[LangchainDocument]:
    df = pd.read_csv(csv_path)
    return [
        LangchainDocument(page_content=row['content'], metadata={"source": row['source'], "page": row['page']})
        for _, row in df.iterrows()
    ]

def generate_question(context: str, question_generator) -> str:
    # Generate a question based on the context
    question = question_generator(f"Generate a question based on this text: {context}", max_length=50, num_return_sequences=1)[0]['generated_text']
    return question.strip()

def generate_questions(dataloader: DataLoader, num_questions: int) -> List[Tuple[str, str]]:
    question_generator = pipeline("text2text-generation", model="facebook/bart-large-cnn", device=0 if REMOVED_SECRET() else -1)
    questions_and_contexts = []
    for batch in tqdm(dataloader, desc="Generating questions", total=min(len(dataloader), num_questions // BATCH_SIZE + 1)):
        for doc in batch:
            # Split the document content into sentences
            sentences = sent_tokenize(doc.page_content)
            
            # Select a random sentence as context if the document is too long
            if len(sentences) > 3:
                context = ' '.join(sentences[:3])  # Use first 3 sentences as context
            else:
                context = doc.page_content
            
            # Generate a question based on the context
            question = generate_question(context, question_generator)
            
            questions_and_contexts.append((question, context))
            
            if len(questions_and_contexts) >= num_questions:
                return questions_and_contexts
    
    return questions_and_contexts

def main():
    # Convert PDFs to CSV
    print("Converting PDFs to CSV...")
    convert_pdfs_to_csv(PDF_FOLDER_PATH, CSV_PATH)

    # Load processed documents
    processed_documents = load_from_csv(CSV_PATH)

    # Create dataset and dataloader
    document_dataset = DocumentDataset(processed_documents)
    document_dataloader = DataLoader(document_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn)

    # Generate questions
    print("Generating questions...")
    questions_and_contexts = generate_questions(document_dataloader, NUM_QUESTIONS)

    # Save questions to CSV
    questions_df = pd.DataFrame(questions_and_contexts, columns=['question', 'context'])
    questions_df.to_csv(QUESTIONS_PATH, index=False, quoting=1)  # quoting=1 ensures that all fields are quoted
    print(f"Generated questions saved to {QUESTIONS_PATH}")

if __name__ == "__main__":
    main()

Converting PDFs to CSV...


Loading PDFs: 100%|██████████| 30/30 [00:03<00:00,  8.73it/s]


Documents: [Document(page_content="Question:\n\nwho owns st andrews golf course in scotland\n\nAnswer:\n\n<P> The Old Course at St Andrews is considered the oldest golf course in the world , a public\n\ncourse over common land in St Andrews , Fife , Scotland . It is held in trust by The St Andrews\n\nLinks Trust under an act of Parliament . The Royal and Ancient Golf Club of St Andrews club house\n\nsits adjacent to the first tee , although it is but one of many clubs that have playing privileges on\n\nthe course , along with the general public . </P>\n\nContext:\n\nOld Course at St Andrews - wikipedia Old Course at St Andrews Jump to :\n\nnavigation , search Old Course R&A Clubhouse and 18th green in 2004 Club\n\ninformation\n\nCoordinates\n\n56 ° 20\n\n35 ''\n\nN 2 ° 48\n\n11 ''\n\n56.343 °\n\nN\n\n2.803\n\n°\n\n/ 56.343 ; - 2.803 Coordinates : 56 ° 20 35 '' N 2 ° 48 11 ''\n\nW\n\n56.343\n\n°\n\nN\n\n2.803\n\n°\n\n/ 56.343 ; - 2.803 Location St Andrews ,\n\nW\n\n/\n\nW\n\n/\n\nW\n\nS

Splitting Documents: 100%|██████████| 30/30 [00:00<00:00, 190.40it/s]


CSV file saved to processed_pdfs.csv
Generating questions...


/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
Generating questions:   0%|          | 0/7 [00:00<?, ?it/s]/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1282: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
Generating questions: 100%|██████████| 7/7 [17:48<00:00, 152.62s/it]

Generated questions saved to generated_questions.csv


nnew stuff down here

In [1]:
import os
import pandas as pd
from tqdm import tqdm
from typing import List, Tuple
from RAG_UTILS import DocumentProcessor, RAGSystem, LangchainDocument
import logging
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
MODEL_ID = "nvidia/Mistral-NeMo-Minitron-8B-Base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
PDF_FOLDER_PATH = "nq_samples_good"
PROCESSED_DATA_PATH = "processed_data.csv"
NUM_RETRIEVED_DOCS = 5

def extract_question_and_answer(content: str) -> Tuple[str, str]:
    lines = content.split('\n')
    question = ""
    answer = ""
    for i, line in enumerate(lines):
        if line.startswith("Question:"):
            question = lines[i+1].strip()
        elif line.startswith("Answer:"):
            answer = ' '.join(lines[i+1:]).strip()
            break
    return question, answer

def process_pdfs(folder_path: str) -> List[Tuple[str, str, str]]:
    doc_processor = DocumentProcessor(EMBEDDING_MODEL_NAME)
    raw_documents = doc_processor.load_pdfs_from_folder(folder_path)
    processed_data = []

    if not raw_documents:
        logging.error(f"No documents were loaded from {folder_path}")
        return []

    for doc in tqdm(raw_documents, desc="Processing PDFs"):
        question, answer = extract_question_and_answer(doc.page_content)
        if question and answer:
            processed_data.append((question, answer, doc.page_content))
        else:
            logging.warning(f"No question or answer found in document: {REMOVED_SECRET('source', 'Unknown')}")

    if not processed_data:
        logging.error("No valid questions and answers were extracted from the documents")

    return processed_data

def save_processed_data(data: List[Tuple[str, str, str]], output_path: str):
    df = pd.DataFrame(data, columns=['question', 'answer', 'context'])
    df.to_csv(output_path, index=False)
    logging.info(f"Processed data saved to {output_path}")

def load_processed_data(file_path: str) -> List[Tuple[str, str, str]]:
    df = pd.read_csv(file_path)
    return list(df.itertuples(index=False, name=None))

def evaluate_rag_system(rag_system: RAGSystem, questions_and_answers: List[Tuple[str, str, str]]):
    results = []
    for question, answer, context in tqdm(questions_and_answers, desc="Evaluating RAG system"):
        rag_answer, relevant_docs, _ = rag_system.answer_with_rag(question, rag_system.knowledge_index, num_retrieved_docs=NUM_RETRIEVED_DOCS)
        results.append({
            "question": question,
            "ground_truth": answer,
            "rag_answer": rag_answer,
            "relevant_docs": [doc.page_content for doc in relevant_docs]
        })
    return results

def main():
    # Process PDFs and extract questions/answers
    if not REMOVED_SECRET(PROCESSED_DATA_PATH):
        processed_data = process_pdfs(PDF_FOLDER_PATH)
        if not processed_data:
            logging.error("No data to process. Exiting.")
            return
        save_processed_data(processed_data, PROCESSED_DATA_PATH)
    else:
        processed_data = load_processed_data(PROCESSED_DATA_PATH)
        if not processed_data:
            logging.error("No data found in the processed data file. Exiting.")
            return

    # Initialize RAG system
    rag_system = RAGSystem(
        embedding_model_name=EMBEDDING_MODEL_NAME,
        model_id=MODEL_ID,
        reranker_model=RERANKER_MODEL,
    )

    # Build knowledge index
    documents = [LangchainDocument(page_content=context, metadata={}) for _, _, context in processed_data]
    if not documents:
        logging.error("No documents to build the knowledge index. Exiting.")
        return
    
    try:
        rag_system.knowledge_index = rag_system.build_vector_database(documents)
    except Exception as e:
        logging.error(f"Error building knowledge index: {str(e)}")
        return

    # Evaluate RAG system
    evaluation_results = evaluate_rag_system(rag_system, processed_data)

    # Save evaluation results
    results_df = pd.DataFrame(evaluation_results)
    results_df.to_csv("rag_evaluation_results.csv", index=False)
    logging.info("Evaluation complete. Results saved to 'rag_evaluation_results.csv'")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt_tab to /home/obb/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/obb/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
Loading PDFs: 100%|██████████| 30/30 [00:02<00:00, 14.65it/s]


Documents: [Document(page_content='Question: fish appeared in the fossil record during the\n\nAnswer: wiki/Cambrian_explosion" title="Cambrian explosion">Cambrian explosion. It was d\n\nContext:\n\nution of fish began about 530 million years ago during the Cambrian explosion. It was during this time that the early chordates developed the skull and the vertebral column, leading to the first craniates and vertebrates. The first fish lineages belong to the Agnatha, or jawless fish. Early examples include Haikouichthys. During the late Cambrian, eel-like jawless fish called the conodonts, and small mostly armoured fish known as ostracoderms, first appeared. Most jawless fish are now extinct; but the extant lampreys may approximate ancient pre-jawed fish. Lampreys belong to the Cyclostomata, which includes the extant hagfish, and this group may have split early on from other agnathans. The first\n\nFull Document (Truncated):\n\nEvolution of fish - Wikipedia .referencetooltip{position:absolu

Processing PDFs:   0%|          | 0/30 [00:00<?, ?it/s]2024-09-20 13:52:52,178 - WARNING - No question or answer found in document: nq_samples_good/nq_sample_28.pdf
2024-09-20 13:52:52,179 - WARNING - No question or answer found in document: nq_samples_good/nq_sample_13.pdf
2024-09-20 13:52:52,179 - WARNING - No question or answer found in document: nq_samples_good/nq_sample_7.pdf
2024-09-20 13:52:52,180 - WARNING - No question or answer found in document: nq_samples_good/nq_sample_24.pdf
2024-09-20 13:52:52,180 - WARNING - No question or answer found in document: nq_samples_good/nq_sample_23.pdf
2024-09-20 13:52:52,180 - WARNING - No question or answer found in document: nq_samples_good/nq_sample_2.pdf
2024-09-20 13:52:52,181 - WARNING - No question or answer found in document: nq_samples_good/nq_sample_26.pdf
2024-09-20 13:52:52,181 - WARNING - No question or answer found in document: nq_samples_good/nq_sample_4.pdf
2024-09-20 13:52:52,181 - WARNING - No question or answer found in d

In [1]:
# RAG System Evaluation Notebook (Updated with Knowledge Index Fix)

import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from rouge_score import rouge_scorer

from REMOVED_SECRET import sentence_bleu, SmoothingFunction
from bert_score import BERTScorer
import os
import re
import fitz  # PyMuPDF

# Import your RAG system
from RAG_UTILS import RAGSystem, EMBEDDING_MODEL_NAME, MODEL_ID, RERANKER_MODEL

MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"
RERANKER_MODEL = None
NUM_RETRIEVED_DOCS = 5

def extract_data_from_pdf(pdf_path):
    """
    Extract question, context, and answer from a PDF file using PyMuPDF.
    """
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    
    # Regex patterns to extract information
    question_pattern = r"Question: (.*?)\n"
    context_pattern = r"Context:(.*?)(?=Question:|$)"
    answer_pattern = r"Answer: (.*?)(?=\n\n|$)"

    questions = re.findall(question_pattern, text)
    contexts = re.findall(context_pattern, text, re.DOTALL)
    answers = re.findall(answer_pattern, text, re.DOTALL)

    # Clean and pair the extracted data
    evaluation_data = []
    for q, c, a in zip(questions, contexts, answers):
        evaluation_data.append({
            "question": q.strip(),
            "context": c.strip(),
            "ground_truth": a.strip()
        })

    return evaluation_data

def calculate_perplexity(logits, input_ids):
    # Ensure logits and input_ids have the same sequence length
    seq_len = min(logits.size(1), input_ids.size(1))
    logits = logits[:, :seq_len, :]
    input_ids = input_ids[:, :seq_len]
    
    # Calculate loss
    loss_fct = REMOVED_SECRET(ignore_index=-100, reduction='none')
    loss = loss_fct(logits.view(-1, logits.size(-1)), input_ids.view(-1))
    
    # Calculate perplexity
    return torch.exp(loss.mean())

def calculate_retrieval_accuracy(retrieved_docs, ground_truth_context, k=1):
    relevant_docs = [doc for doc in retrieved_docs[:k] if ground_truth_context in doc]
    return len(relevant_docs) / k

def calculate_bleu_score(reference, hypothesis):
    """
    Calculate BLEU score with smoothing and lower n-gram order.
    """
    smoothie = SmoothingFunction().method1
    return sentence_bleu([reference.split()], hypothesis.split(), 
                         weights=(0.5, 0.5), 
                         smoothing_function=smoothie)

def evaluate_rag_system(rag_system, evaluation_data, pdf_folder_path):
    results = []
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    
    # Load and process documents
    raw_documents = REMOVED_SECRET(pdf_folder_path)
    processed_documents = REMOVED_SECRET(raw_documents)

    # Build vector database
    knowledge_index = rag_system.build_vector_database(processed_documents)
    
    for sample in tqdm(evaluation_data, desc="Evaluating samples"):
        question = sample['question']
        ground_truth = sample['ground_truth']
        context = sample['context']
        
        # Get RAG system's answer and relevant documents
        answer, relevant_docs, logits = rag_system.answer_with_rag(question, knowledge_index)
        
        # Calculate metrics
        bleu_score = calculate_bleu_score(ground_truth, answer)
        rouge_scores = rouge_scorer_instance.score(ground_truth, answer)
        retrieval_accuracy = calculate_retrieval_accuracy(relevant_docs, context)
        
        # BERT Score
        _, _, bert_f1 = bert_scorer.score([answer], [ground_truth])
        
        # Perplexity calculation
        input_ids = REMOVED_SECRET.encode(question + answer, return_tensors="pt").to(logits.device)
        perplexity = calculate_perplexity(logits, input_ids)
        
        results.append({
            'question': question,
            'ground_truth': ground_truth,
            'generated_answer': answer,
            'bleu_score': bleu_score,
            'rouge1': rouge_scores['rouge1'].fmeasure,
            'rouge2': rouge_scores['rouge2'].fmeasure,
            'rougeL': rouge_scores['rougeL'].fmeasure,
            'retrieval_accuracy': retrieval_accuracy,
            'bert_score': bert_f1.item(),
            'perplexity': perplexity.item()
        })
    
    return pd.DataFrame(results)

# Function to plot distribution of scores
def plot_score_distribution(data, score_name):
    plt.figure(figsize=(10, 6))
    sns.histplot(data[score_name], kde=True)
    plt.title(f'Distribution of {score_name}')
    plt.xlabel(score_name)
    plt.ylabel('Frequency')
    plt.show()

# Function for error analysis
def error_analysis(results):
    # Calculate absolute difference between BLEU score and 1 (perfect score)
    results['bleu_error'] = 1 - results['bleu_score']
    
    # Sort by error and get top 10 worst predictions
    worst_predictions = results.sort_values('bleu_error', ascending=False).head(10)
    
    print("Top 10 Worst Predictions:")
    for _, row in worst_predictions.iterrows():
        print(f"Question: {row['question']}")
        print(f"Ground Truth: {row['ground_truth']}")
        print(f"Generated Answer: {row['generated_answer']}")
        print(f"BLEU Score: {row['bleu_score']}")
        print("--------------------")

    # Analyze error patterns (this is a simple example, you might want to expand this)
    error_patterns = {
        'short_answer': (results['generated_answer'].str.split().str.len() < 5).sum(),
        'long_answer': (results['generated_answer'].str.split().str.len() > 50).sum(),
        'low_bleu': (results['bleu_score'] < 0.1).sum()
    }
    
    print("\nError Patterns:")
    for pattern, count in error_patterns.items():
        print(f"{pattern}: {count}")

# Main execution
if __name__ == "__main__":
    # Initialize RAG system
    rag_system = RAGSystem(
        embedding_model_name=EMBEDDING_MODEL_NAME,
        model_id=MODEL_ID,
        reranker_model=RERANKER_MODEL,
    )

    # Extract data from all PDFs in a directory
    pdf_directory = "nq_samples_good"  # Replace with your PDF directory path
    all_evaluation_data = []
    for filename in os.listdir(pdf_directory):
        if filename.endswith(".pdf"):
            pdf_path = REMOVED_SECRET(pdf_directory, filename)
            all_evaluation_data.extend(extract_data_from_pdf(pdf_path))

    # Run evaluation
    evaluation_results = evaluate_rag_system(rag_system, all_evaluation_data, pdf_directory)

    # Display results
    print(evaluation_results.describe())

    # Calculate average scores for numeric columns only
    numeric_columns = evaluation_results.select_dtypes(include=[np.number]).columns
    average_scores = evaluation_results[numeric_columns].mean()
    print("\nAverage Scores:")
    print(average_scores)

    # Plot distributions for numeric columns
    for metric in numeric_columns:
        plot_score_distribution(evaluation_results, metric)

    # Correlation heatmap for numeric columns
    plt.figure(figsize=(12, 10))
    sns.heatmap(evaluation_results[numeric_columns].corr(), annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap of Evaluation Metrics')
    plt.show()

    # Error analysis
    error_analysis(evaluation_results)

    # Display generated answers
    print("\nGenerated Answers:")
    for _, row in evaluation_results.iterrows():
        print(f"Question: {row['question']}")
        print(f"Ground Truth: {row['ground_truth']}")
        print(f"Generated Answer: {row['generated_answer']}")
        print("--------------------")

    # Clear memory
    rag_system.clear_memory()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['REMOVED_SECRET.bias', 'REMOVED_SECRET.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading PDFs: 100%|██████████| 30/30 [00:01<00:00, 15.72it/s]


Documents: [Document(page_content='Question: fish appeared in the fossil record during the\n\nAnswer: wiki/Cambrian_explosion" title="Cambrian explosion">Cambrian explosion. It was d\n\nContext:\n\nution of fish began about 530 million years ago during the Cambrian explosion. It was during this time that the early chordates developed the skull and the vertebral column, leading to the first craniates and vertebrates. The first fish lineages belong to the Agnatha, or jawless fish. Early examples include Haikouichthys. During the late Cambrian, eel-like jawless fish called the conodonts, and small mostly armoured fish known as ostracoderms, first appeared. Most jawless fish are now extinct; but the extant lampreys may approximate ancient pre-jawed fish. Lampreys belong to the Cyclostomata, which includes the extant hagfish, and this group may have split early on from other agnathans. The first\n\nFull Document (Truncated):\n\nEvolution of fish - Wikipedia .referencetooltip{position:absolu

Evaluating samples:   0%|          | 0/30 [00:00<?, ?it/s]

=> Retrieving documents...
Retrieved 30 documents.
Using 1 documents for answering.
=> Generating answer...


...

Evaluating samples:  97%|█████████▋| 29/30 [09:14<00:17, 17.33s/it]

=> Retrieving documents...
Retrieved 30 documents.
Using 1 documents for answering.
=> Generating answer...


Evaluating samples: 100%|██████████| 30/30 [09:41<00:00, 19.37s/it]

         bleu_score     rouge1     rouge2     rougeL  retrieval_accuracy  \
count  3.000000e+01  30.000000  30.000000  30.000000                30.0   
mean   1.827483e-04   0.064890   0.037563   0.056051                 0.0   
std    6.566587e-04   0.016701   0.020128   0.017055                 0.0   
min    1.572996e-13   0.040346   0.000000   0.023346                 0.0   
25%    7.999858e-07   0.054702   0.027517   0.043845                 0.0   
50%    6.610064e-06   0.060769   0.037856   0.055711                 0.0   
75%    7.454518e-05   0.073871   0.049646   0.065322                 0.0   
max    3.566894e-03   0.110236   0.090395   0.101124                 0.0   

       bert_score    perplexity  
count   30.000000  3.000000e+01  
mean    -0.327708  4.056559e+05  
std      0.110857  6.841960e+05  
min     -0.624531  1.067466e+04  
25%     -0.394986  1.111732e+05  
50%     -0.298341  1.858772e+05  
75%     -0.261672  3.683714e+05  
max     -0.124858  3.485244e+06  

Average 

Top 10 Worst Predictions:
Question: where was donovan mitchell picked in the draft
Ground Truth: i/20
Context:
="/wiki/Denver_Nuggets" title="Denver Nuggets">Denver Nuggets with the 13th overall pick in the 2017
NBA draft only to be traded to the Utah Jazz for the 24th pick (Tyler Lydon) and Trey Lyles.[15] On July
5, 2017, Mitchell signed a four-year rookie scale contract with the Jazz.[16] On July 11, 2017, Mitchell
signed a multi-year shoe deal with Adidas. Later that day, Mitchell scored 37 points against the
Memphis Grizzlies in the 2017 NBA Summer League in Las Vegas, the most by any player during the
2017 NBA Summer League.[17] In his NBA debut on October 18, 2017, Mitchell registered 10 points
and 4 assists against the Denver Nuggets.[18] On December 1, 2017, he scored a career-high 41
points in a 114–108 win over the New Orleans Pelicans. He set the Jazz scoring record for a rookie and
became the first NBA rookie to score 40 points in a game since Blake Griffin in 2011.[19] He

decent v

In [1]:
# RAG System Evaluation Notebook (Improved)

import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from rouge_score import rouge_scorer
from REMOVED_SECRET import sentence_bleu, SmoothingFunction
from bert_score import BERTScorer
import os
import re
import fitz  # PyMuPDF

# Import your RAG system
from RAG_UTILS import RAGSystem, EMBEDDING_MODEL_NAME, MODEL_ID, RERANKER_MODEL

#MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"
RERANKER_MODEL = None
NUM_RETRIEVED_DOCS = 5

def clean_text(text: str) -> str:
    """Clean and normalize text."""
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text.lower().strip()

def is_valid_answer(answer: str) -> bool:
    """Check if an answer is valid."""
    cleaned = clean_text(answer)
    return len(cleaned) > 1 and not cleaned.isdigit()  # Adjust criteria as needed

def extract_data_from_pdf(pdf_path):
    """
    Extract question, context, and answer from a PDF file using PyMuPDF.
    """
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    
    # Regex patterns to extract information
    question_pattern = r"Question: (.*?)\n"
    context_pattern = r"Context:(.*?)(?=Question:|$)"
    answer_pattern = r"Answer: (.*?)(?=\n\n|$)"

    questions = re.findall(question_pattern, text)
    contexts = re.findall(context_pattern, text, re.DOTALL)
    answers = re.findall(answer_pattern, text, re.DOTALL)

    # Clean and pair the extracted data
    evaluation_data = []
    for q, c, a in zip(questions, contexts, answers):
        evaluation_data.append({
            "question": q.strip(),
            "context": c.strip(),
            "ground_truth": a.strip()
        })

    return evaluation_data

def calculate_perplexity(logits, input_ids):
    # Ensure logits and input_ids have the same sequence length
    seq_len = min(logits.size(1), input_ids.size(1))
    logits = logits[:, :seq_len, :]
    input_ids = input_ids[:, :seq_len]
    
    # Calculate loss
    loss_fct = REMOVED_SECRET(ignore_index=-100, reduction='none')
    loss = loss_fct(logits.view(-1, logits.size(-1)), input_ids.view(-1))
    
    # Calculate perplexity
    return torch.exp(loss.mean())

def calculate_retrieval_accuracy(retrieved_docs, ground_truth_context, k=1):
    relevant_docs = [doc for doc in retrieved_docs[:k] if ground_truth_context in doc]
    return len(relevant_docs) / k

def calculate_bleu_score(reference, hypothesis):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference.split()], hypothesis.split(), 
                         weights=(0.25, 0.25, 0.25, 0.25), 
                         smoothing_function=smoothie)

def evaluate_rag_system(rag_system, evaluation_data, pdf_folder_path):
    results = []
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    
    # Load and process documents
    raw_documents = REMOVED_SECRET(pdf_folder_path)
    processed_documents = REMOVED_SECRET(raw_documents)

    # Build vector database
    knowledge_index = rag_system.build_vector_database(processed_documents)
    
    for sample in tqdm(evaluation_data, desc="Evaluating samples"):
        question = sample['question']
        ground_truth = sample['ground_truth']
        context = sample['context']
        
        if not is_valid_answer(ground_truth):
            print(f"Warning: Invalid ground truth for question: {question}")
            continue
        
        # Get RAG system's answer and relevant documents
        answer, relevant_docs, logits = rag_system.answer_with_rag(question, knowledge_index)
        
        # Calculate metrics
        bleu_score = calculate_bleu_score(ground_truth, answer)
        rouge_scores = rouge_scorer_instance.score(ground_truth, answer)
        retrieval_accuracy = calculate_retrieval_accuracy(relevant_docs, context)
        
        # BERT Score
        _, _, bert_f1 = bert_scorer.score([answer], [ground_truth])
        
        # Perplexity calculation
        input_ids = REMOVED_SECRET.encode(question + answer, return_tensors="pt").to(logits.device)
        perplexity = calculate_perplexity(logits, input_ids)
        
        results.append({
            'question': question,
            'ground_truth': ground_truth,
            'generated_answer': answer,
            'bleu_score': bleu_score,
            'rouge1': rouge_scores['rouge1'].fmeasure,
            'rouge2': rouge_scores['rouge2'].fmeasure,
            'rougeL': rouge_scores['rougeL'].fmeasure,
            'retrieval_accuracy': retrieval_accuracy,
            'bert_score': bert_f1.item(),
            'perplexity': perplexity.item()
        })
    
    return pd.DataFrame(results)

# Function to plot distribution of scores
def plot_score_distribution(data, score_name):
    plt.figure(figsize=(10, 6))
    sns.histplot(data[score_name], kde=True)
    plt.title(f'Distribution of {score_name}')
    plt.xlabel(score_name)
    plt.ylabel('Frequency')
    plt.show()

# Function for error analysis
def error_analysis(results):
    # Calculate absolute difference between BLEU score and 1 (perfect score)
    results['bleu_error'] = 1 - results['bleu_score']
    
    # Sort by error and get top 10 worst predictions
    worst_predictions = results.sort_values('bleu_error', ascending=False).head(10)
    
    print("Top 10 Worst Predictions:")
    for _, row in worst_predictions.iterrows():
        print(f"Question: {row['question']}")
        print(f"Ground Truth: {row['ground_truth']}")
        print(f"Generated Answer: {row['generated_answer']}")
        print(f"BLEU Score: {row['bleu_score']}")
        print("--------------------")

    # Analyze error patterns (this is a simple example, you might want to expand this)
    error_patterns = {
        'short_answer': (results['generated_answer'].str.split().str.len() < 5).sum(),
        'long_answer': (results['generated_answer'].str.split().str.len() > 50).sum(),
        'low_bleu': (results['bleu_score'] < 0.1).sum()
    }
    
    print("\nError Patterns:")
    for pattern, count in error_patterns.items():
        print(f"{pattern}: {count}")

# Main execution
if __name__ == "__main__":
    # Initialize RAG system
    rag_system = RAGSystem(
        embedding_model_name=EMBEDDING_MODEL_NAME,
        model_id=MODEL_ID,
        reranker_model=RERANKER_MODEL,
    )

    # Extract data from all PDFs in a directory
    pdf_directory = "local_database"  # Replace with your PDF directory path
    all_evaluation_data = []
    for filename in os.listdir(pdf_directory):
        if filename.endswith(".pdf"):
            pdf_path = REMOVED_SECRET(pdf_directory, filename)
            all_evaluation_data.extend(extract_data_from_pdf(pdf_path))

    # Run evaluation
    evaluation_results = evaluate_rag_system(rag_system, all_evaluation_data, pdf_directory)

    # Display results
    print(evaluation_results.describe())

    # Calculate average scores for numeric columns only
    numeric_columns = evaluation_results.select_dtypes(include=[np.number]).columns
    average_scores = evaluation_results[numeric_columns].mean()
    print("\nAverage Scores:")
    print(average_scores)

    # Plot distributions for numeric columns
    for metric in numeric_columns:
        plot_score_distribution(evaluation_results, metric)

    # Correlation heatmap for numeric columns
    plt.figure(figsize=(12, 10))
    sns.heatmap(evaluation_results[numeric_columns].corr(), annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap of Evaluation Metrics')
    plt.show()

    # Error analysis
    error_analysis(evaluation_results)

    # Display generated answers
    print("\nGenerated Answers:")
    for _, row in evaluation_results.iterrows():
        print(f"Question: {row['question']}")
        print(f"Ground Truth: {row['ground_truth']}")
        print(f"Generated Answer: {row['generated_answer']}")
        print(f"BLEU Score: {row['bleu_score']:.4f}")
        print("--------------------")

    # Clear memory
    rag_system.clear_memory()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['REMOVED_SECRET.bias', 'REMOVED_SECRET.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading PDFs: 100%|██████████| 31/31 [00:01<00:00, 15.64it/s]


Documents: [Document(page_content="Question: fish appeared in the fossil record during the\n\nAnswer: During the Cambrian Explosion.\n\nContext:\n\nution of fish began about 530 million years ago during the Cambrian explosion. It was during this time that the early chordates developed the skull and the vertebral column, leading to the first craniates and vertebrates. The first fish lineages belong to the Agnatha, or jawless fish. Early examples include Haikouichthys. During the late Cambrian, eel-like jawless fish called the conodonts, and small mostly armoured fish known as ostracoderms, first appeared. Most jawless fish are now extinct; but the extant lampreys may approximate ancient pre-jawed fish. Lampreys belong to the Cyclostomata, which includes the extant hagfish, and this group may have split early on from other agnathans. The first\n\nFull Document (Truncated):\n\nEvolution of fish - Wikipedia .referencetooltip{position:absolute;list-style:none;list-style-image:none;opa li{ba

Evaluating samples:   0%|          | 0/30 [00:00<?, ?it/s]

=> Retrieving documents...


...

Evaluating samples:  97%|█████████▋| 29/30 [48:12<01:36, 96.42s/it]

=> Retrieving documents...
Retrieved 30 documents.
Using 1 documents for answering.
=> Generating answer...


Evaluating samples: 100%|██████████| 30/30 [49:46<00:00, 99.56s/it]


       bleu_score     rouge1     rouge2     rougeL  retrieval_accuracy  \
count   30.000000  30.000000  30.000000  30.000000                30.0   
mean     0.002150   0.066734   0.041234   0.059621                 0.0   
std      0.007329   0.040875   0.044916   0.043285                 0.0   
min      0.000000   0.007092   0.000000   0.007092                 0.0   
25%      0.000014   0.034001   0.006526   0.028804                 0.0   
50%      0.000104   0.061584   0.017314   0.043025                 0.0   
75%      0.000328   0.083276   0.071800   0.083276                 0.0   
max      0.034318   0.169014   0.152000   0.169014                 0.0   

       bert_score    perplexity  
count   30.000000  3.000000e+01  
mean    -0.323116  3.794850e+05  
std      0.134919  5.425822e+05  
min     -0.610538  6.133597e+03  
25%     -0.400268  1.011464e+05  
50%     -0.369106  1.784127e+05  
75%     -0.221593  4.913322e+05  
max     -0.062362  2.804353e+06  

Average Scores:
bleu_score

Top 10 Worst Predictions:
Question: when does life is strange before the storm part 2
Ground Truth: October 2017
Context:
Life Is Strange: Before the Storm Developer(s) Deck Nine[a] Publisher(s) Square Enix Director(s) Webb
Pickersgill Chris Floyd Producer(s) David Lawrence Hein Zoe Brown Designer(s) William Beacham
Programmer(s) Mark Lyons Danielle Cheah Artist(s) Andrew Weatherl Writer(s) Zak Garriss
Composer(s) Daughter Engine Unity Platform(s) Microsoft Windows PlayStation 4 Xbox One Release
Episode 1 WW: 31 August 2017 Episode 2 WW: 19 October 2017 Episode 3 WW: 20 December 2017
Genre(s) Graphic adventure Mode(s) Single-player
Full Document (Truncated):
Life Is Strange: Before the Storm - Wikipedia .referencetooltip{position:absolute;list-style:none;list-style
-image:none;opacity:0;font-size:10px;margin:0;z-index:5;padding:0}.referencetooltip li{border:#080086
2px 
solid;max-width:260px;padding:10px 
8px 
13px
8px;margin:0px;background-color:#F7F7F7;-webkit-box-shadow:2px 
4px 
2p

GREAT V

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from umap import UMAP

def create_animated_umap(embeddings, labels, n_neighbors=15, min_dist=0.1, n_components=3, metric='cosine', n_frames=100):
    # Normalize the embeddings
    scaler = StandardScaler()
    scaled_embeddings = scaler.fit_transform(embeddings)

    # Create UMAP reducer
    reducer = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, metric=metric)

    # Fit UMAP
    umap_embeddings = reducer.fit_transform(scaled_embeddings)

    # Create interpolation between initial state and final UMAP embedding
    initial_state = scaled_embeddings[:, :n_components]  # Use first n_components of scaled embeddings
    embedding_list = [
        initial_state + (umap_embeddings - initial_state) * (i / (n_frames - 1))
        for i in range(n_frames)
    ]

    # Set up the figure and 3D axis
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Set style and color palette
    REMOVED_SECRET("dark_background")
    color_palette = sns.color_palette("plasma", n_colors=len(np.unique(labels)))

    # Create scatter plot
    scatter = ax.scatter(
        embedding_list[0][:, 0],
        embedding_list[0][:, 1],
        embedding_list[0][:, 2],
        c=labels,
        cmap=REMOVED_SECRET("plasma"),
        s=20
    )
    
    # Add title and labels
    title = ax.set_title("UMAP Embedding Progress (Frame 0)", fontsize=16, color='cyan')
    ax.set_xlabel("UMAP1", fontsize=12, color='magenta')
    ax.set_ylabel("UMAP2", fontsize=12, color='magenta')
    ax.set_zlabel("UMAP3", fontsize=12, color='magenta')

    # Add a color bar
    cbar = fig.colorbar(scatter, ax=ax, pad=0.1)
    cbar.set_label("Document Clusters", fontsize=12, color='yellow')

    # Set consistent axis limits
    all_embeddings = np.vstack(embedding_list)
    ax.set_xlim(all_embeddings[:, 0].min(), all_embeddings[:, 0].max())
    ax.set_ylim(all_embeddings[:, 1].min(), all_embeddings[:, 1].max())
    ax.set_zlim(all_embeddings[:, 2].min(), all_embeddings[:, 2].max())

    # Animation update function
    def update(frame):
        title.set_text(f"UMAP Embedding Progress (Frame {frame})")
        scatter._offsets3d = (embedding_list[frame][:, 0], 
                              embedding_list[frame][:, 1], 
                              embedding_list[frame][:, 2])
        return scatter, title

    # Create animation
    anim = animation.FuncAnimation(fig, update, frames=n_frames, interval=50, blit=False)

    # Save animation
    anim.save('umap_progress_animation.gif', writer='pillow', fps=30)

    plt.close(fig)

    print("UMAP progress animation saved as 'umap_progress_animation.gif'")

# Usage example:
# embeddings = np.array([...])  # Your document embeddings
# labels = np.array([...])  # Cluster labels or document types
# create_animated_umap(embeddings, labels)

In [2]:
import plotly.graph_objects as go
import numpy as np

def create_text_splitting_sankey(documents, chunks):
    # Create lists for Sankey diagram
    source = []
    target = []
    value = []
    label = []

    # Add documents
    for i, doc in enumerate(documents):
        label.append(f"Doc {i+1}")
        doc_chunks = [chunk for chunk in chunks if chunk.metadata['source'] == doc.metadata['source']]
        for j, chunk in enumerate(doc_chunks):
            source.append(i)
            target.append(len(documents) + len(source) - 1)
            value.append(len(chunk.page_content))
            label.append(f"Chunk {j+1}")

    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = label,
          color = "blue"
        ),
        link = dict(
          source = source,
          target = target,
          value = value
    ))])

    fig.update_layout(title_text="Document Splitting Process", font_size=10)
    fig.write_html("text_splitting_sankey.html")
    print("Text splitting Sankey diagram saved as 'text_splitting_sankey.html'")

# Usage:
# create_text_splitting_sankey(documents, chunks)

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
from typing import List
import math

def visualize_retrieval(question: str, relevant_docs: List[str], similarity_scores: List[float], file_name: str):
    """
    Creates and saves a tree-like retrieval graph for a given question and its relevant documents.
    
    Args:
    question (str): The query question.
    relevant_docs (List[str]): List of retrieved documents.
    similarity_scores (List[float]): Similarity scores for each document.
    file_name (str): Name of the file to save the graph.
    """
    G = nx.Graph()

    # Add query node
    G.add_node("Query", pos=(0, 0))

    # Sort documents by similarity score
    sorted_docs = sorted(zip(relevant_docs, similarity_scores), key=lambda x: x[1], reverse=True)

    # Calculate positions for document nodes
    num_docs = len(sorted_docs)
    radius = 10
    angle_step = 2 * math.pi / num_docs

    for i, (doc, score) in enumerate(sorted_docs):
        doc_id = f"Doc {i+1}\n(Score: {score:.2f})"
        angle = i * angle_step
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        G.add_node(doc_id, pos=(x, y))
        G.add_edge("Query", doc_id, weight=score)

    # Set up the plot
    plt.figure(figsize=(15, 15))
    pos = nx.get_node_attributes(G, 'pos')

    # Draw edges
    edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
    nx.draw_networkx_edges(G, pos, width=edge_weights, edge_color='lightgray', arrows=True, arrowsize=20)

    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='lightblue', alpha=0.8)
    nx.draw_networkx_nodes(G, pos, nodelist=["Query"], node_size=5000, node_color='lightgreen', alpha=0.8)

    # Add labels
    nx.draw_networkx_labels(G, pos, font_size=8, font_weight="bold")

    # Add document content as text
    for node, (x, y) in pos.items():
        if node != "Query":
            doc_index = int(node.split()[1]) - 1
            doc_content = sorted_docs[doc_index][0]
            if isinstance(doc_content, str):
                doc_text = doc_content[:100]  # First 100 characters
            elif hasattr(doc_content, 'page_content'):
                doc_text = doc_content.page_content[:100]
            else:
                doc_text = str(doc_content)[:100]
            plt.text(x, y-2, doc_text, wrap=True, ha='center', va='center', fontsize=6, bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))

    plt.title(f"Retrieval Tree for Query: {question[:50]}...", fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(file_name, dpi=300, bbox_inches='tight')
    plt.close()

# Example usage:
# visualize_retrieval(question, relevant_docs, similarity_scores, f"retrieval_tree_{i}.png")

In [4]:
# Improved RAG System Evaluation Script

import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from rouge_score import rouge_scorer
from REMOVED_SECRET import sentence_bleu, SmoothingFunction
from bert_score import BERTScorer
import os
import re
import fitz  # PyMuPDF
from collections import Counter
from difflib import SequenceMatcher
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict
from REMOVED_SECRET import cosine_similarity
# Import your RAG system
from RAG_UTILS import RAGSystem, EMBEDDING_MODEL_NAME, MODEL_ID, RERANKER_MODEL

MODEL_ID = "microsoft/Phi-3.5-mini-instruct"
RERANKER_MODEL = None
NUM_RETRIEVED_DOCS = 5

def clean_text(text: str) -> str:
    """Clean and normalize text."""
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text.lower().strip()

def is_valid_answer(answer: str) -> bool:
    """Check if an answer is valid."""
    cleaned = clean_text(answer)
    return len(cleaned) > 1 and not cleaned.isdigit()  # Adjust criteria as needed

def extract_data_from_pdf(pdf_path):
    """
    Extract question, context, and answer from a PDF file using PyMuPDF.
    """
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    
    # Regex patterns to extract information
    question_pattern = r"Question: (.*?)\n"
    context_pattern = r"Context:(.*?)(?=Question:|Answer:|$)"
    answer_pattern = r"Answer: (.*?)(?=Context:|$)"

    questions = re.findall(question_pattern, text)
    contexts = re.findall(context_pattern, text, re.DOTALL)
    answers = re.findall(answer_pattern, text, re.DOTALL)

    # Clean and pair the extracted data
    evaluation_data = []
    for q, c, a in zip(questions, contexts, answers):
        evaluation_data.append({
            "question": q.strip(),
            "context": c.strip(),
            "ground_truth": a.strip()
        })

    return evaluation_data

def calculate_perplexity(logits, input_ids):
    # Ensure logits and input_ids have the same sequence length
    seq_len = min(logits.size(1), input_ids.size(1))
    logits = logits[:, :seq_len, :]
    input_ids = input_ids[:, :seq_len]
    
    # Calculate loss
    loss_fct = REMOVED_SECRET(ignore_index=-100, reduction='none')
    loss = loss_fct(logits.view(-1, logits.size(-1)), input_ids.view(-1))
    
    # Calculate perplexity
    return torch.exp(loss.mean())

def calculate_retrieval_accuracy(retrieved_docs, ground_truth_context, k=1):
    def preprocess_text(text):
        return ' '.join(text.lower().split())

    ground_truth_context = preprocess_text(ground_truth_context)
    
    relevant_docs = 0
    for doc in retrieved_docs[:k]:
        # Check if doc is a string or an object with page_content attribute
        if isinstance(doc, str):
            doc_text = preprocess_text(doc)
        else:
            doc_text = preprocess_text(doc.page_content)
        
        # Check for significant overlap
        similarity = SequenceMatcher(None, ground_truth_context, doc_text).ratio()
        if similarity > 0.5:  # You can adjust this threshold
            relevant_docs += 1
            break  # Stop after finding the first relevant document

    return relevant_docs / k

def calculate_bleu_score(reference, hypothesis):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference.split()], hypothesis.split(), 
                         weights=(0.25, 0.25, 0.25, 0.25), 
                         smoothing_function=smoothie)

def normalize_answer(text):
    """
    Normalize answer text while preserving important punctuation and structure.
    """
    # Convert to lowercase
    text = text.lower()
    
    # Replace multiple whitespace characters with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    # Preserve common punctuation that might be important for context
    text = re.sub(r'[^a-z0-9\s.,;:()"-]', '', text)
    
    # Normalize some common variations
    text = text.replace(' , ', ', ').replace(' . ', '. ')
    text = text.replace('( ', '(').replace(' )', ')')
    
    # Remove spaces before punctuation
    text = re.sub(r'\s([.,;:])', r'\1', text)
    
    return text


def exact_match_score(prediction, ground_truth):
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1

def evaluate_rag_system(rag_system, evaluation_data, pdf_folder_path):
    results = []
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    
    # Load and process documents
    raw_documents = REMOVED_SECRET(pdf_folder_path)
    processed_documents = REMOVED_SECRET(raw_documents)

    # Build vector database
    knowledge_index = rag_system.build_vector_database(processed_documents)
    
    for sample in tqdm(evaluation_data, desc="Evaluating samples"):
        question = sample['question']
        ground_truth = normalize_answer(sample['ground_truth'])
        context = sample['context']
        
        print(f"\n\nQuestion: {question}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Original Context: {context[:200]}...")  # Print first 200 characters of context
        
        if not is_valid_answer(ground_truth):
            print(f"Warning: Invalid ground truth for question: {question}")
            continue
        
        # Get RAG system's answer and relevant documents
        answer, relevant_docs, logits = rag_system.answer_with_rag(question, knowledge_index)
        answer = normalize_answer(answer)

        # Calculate similarity scores
        question_embedding = REMOVED_SECRET(question)
        doc_embeddings = REMOVED_SECRET([doc.page_content if hasattr(doc, 'page_content') else str(doc) for doc in relevant_docs])
        similarity_scores = cosine_similarity([question_embedding], doc_embeddings)[0]
        
        print(f"Generated Answer: {answer}")
        print("Retrieved Documents:")
        for i, doc in enumerate(relevant_docs[:3], 1):  # Print top 3 retrieved documents
            print(f"Doc {i}: {doc[:200]}...")  # Print first 200 characters of each document
        
        # Calculate metrics
        bleu_score = calculate_bleu_score(ground_truth, answer)
        rouge_scores = rouge_scorer_instance.score(ground_truth, answer)
        retrieval_accuracy = calculate_retrieval_accuracy(relevant_docs, context)
        
        # BERT Score
        _, _, bert_f1 = bert_scorer.score([answer], [ground_truth])
        
        # Perplexity calculation
        input_ids = REMOVED_SECRET.encode(question + answer, return_tensors="pt").to(logits.device)
        perplexity = calculate_perplexity(logits, input_ids)
        
        # Additional metrics
        exact_match = exact_match_score(answer, ground_truth)
        f1 = f1_score(answer, ground_truth)
        
        print(f"BLEU Score: {bleu_score:.4f}")
        print(f"ROUGE-L F1: {rouge_scores['rougeL'].fmeasure:.4f}")
        print(f"Retrieval Accuracy: {retrieval_accuracy:.4f}")
        print(f"BERT Score: {bert_f1.item():.4f}")
        print(f"Exact Match: {exact_match}")
        print(f"F1 Score: {f1:.4f}")
        
        results.append({
            'question': question,
            'ground_truth': ground_truth,
            'generated_answer': answer,
            'bleu_score': bleu_score,
            'rouge1': rouge_scores['rouge1'].fmeasure,
            'rouge2': rouge_scores['rouge2'].fmeasure,
            'rougeL': rouge_scores['rougeL'].fmeasure,
            'retrieval_accuracy': retrieval_accuracy,
            'bert_score': bert_f1.item(),
            'perplexity': perplexity.item(),
            'exact_match': exact_match,
            'f1_score': f1,
            'relevant_docs': relevant_docs,
            'similarity_scores': similarity_scores.tolist()
        })
    
    return pd.DataFrame(results)
# Function to plot distribution of scores
def plot_score_distribution(data, score_name):
    plt.figure(figsize=(10, 6))
    sns.histplot(data[score_name], kde=True)
    plt.title(f'Distribution of {score_name}')
    plt.xlabel(score_name)
    plt.ylabel('Frequency')
    plt.show()

# Function for error analysis
def error_analysis(results):
    # Calculate absolute difference between BLEU score and 1 (perfect score)
    results['bleu_error'] = 1 - results['bleu_score']
    
    # Sort by error and get top 10 worst predictions
    worst_predictions = results.sort_values('bleu_error', ascending=False).head(10)
    
    print("Top 10 Worst Predictions:")
    for _, row in worst_predictions.iterrows():
        print(f"Question: {row['question']}")
        print(f"Ground Truth: {row['ground_truth']}")
        print(f"Generated Answer: {row['generated_answer']}")
        print(f"BLEU Score: {row['bleu_score']}")
        print(f"F1 Score: {row['f1_score']}")
        print(f"Exact Match: {row['exact_match']}")
        print("--------------------")

    # Analyze error patterns
    error_patterns = {
        'short_answer': (results['generated_answer'].str.split().str.len() < 5).sum(),
        'long_answer': (results['generated_answer'].str.split().str.len() > 50).sum(),
        'low_bleu': (results['bleu_score'] < 0.1).sum(),
        'low_f1': (results['f1_score'] < 0.5).sum(),
        'no_exact_match': (results['exact_match'] == 0).sum()
    }
    
    print("\nError Patterns:")
    for pattern, count in error_patterns.items():
        print(f"{pattern}: {count}")


def prepare_umap_data(rag_system, pdf_directory):
    # Load and process documents
    raw_documents = REMOVED_SECRET(pdf_directory)
    processed_documents = REMOVED_SECRET(raw_documents)

    # Generate embeddings
    embeddings = REMOVED_SECRET([doc.page_content for doc in processed_documents])

    # Convert embeddings to numpy array
    embeddings_array = np.array(embeddings)

    # Create labels based on document sources
    source_to_label = defaultdict(lambda: len(source_to_label))
    labels = np.array([source_to_label[REMOVED_SECRET('source', 'unknown')] for doc in processed_documents])

    # Alternatively, use K-means clustering to create labels
    # n_clusters = min(10, len(processed_documents))  # Adjust the number of clusters as needed
    # kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    # labels = kmeans.fit_predict(embeddings_array)

    return embeddings_array, labels, processed_documents



# Main execution
if __name__ == "__main__":
    # Initialize RAG system
    rag_system = RAGSystem(
        embedding_model_name=EMBEDDING_MODEL_NAME,
        model_id=MODEL_ID,
        reranker_model=RERANKER_MODEL,
    )

    # Extract data from all PDFs in a directory
    pdf_directory = "local_database"  # Replace with your PDF directory path

    # Load and process documents
    raw_documents = REMOVED_SECRET(pdf_directory)
    processed_documents = REMOVED_SECRET(raw_documents)
    # For text splitting visualization
    create_text_splitting_sankey(raw_documents, processed_documents)

    # Prepare data for UMAP visualization
    embeddings, labels, processed_documents = prepare_umap_data(rag_system, pdf_directory)
    

    # Create UMAP visualization
    create_animated_umap(embeddings, labels)
 
    all_evaluation_data = []
    for filename in os.listdir(pdf_directory):
        if filename.endswith(".pdf"):
            pdf_path = REMOVED_SECRET(pdf_directory, filename)
            all_evaluation_data.extend(extract_data_from_pdf(pdf_path))

    # Run evaluation
    evaluation_results = evaluate_rag_system(rag_system, all_evaluation_data, pdf_directory)

    # After evaluation, create visualizations for each result
    for i, result in enumerate(evaluation_results.itertuples()):
        question = result.question
        answer = result.generated_answer
        relevant_docs = result.relevant_docs  # Assuming you've stored this in your results
        similarity_scores = result.similarity_scores  # Assuming you've stored this in your results

        # Create visualization
        visualize_retrieval(question, relevant_docs, similarity_scores, f"retrieval_graph_{i}.png")

        print(f"Created visualization for question {i}: {question[:50]}...")

    # Display some information about the UMAP visualization
    print(f"UMAP visualization created with {len(embeddings)} document chunks.")
    print(f"Number of unique labels: {len(np.unique(labels))}")
    print("Label distribution:")
    for label, count in zip(*np.unique(labels, return_counts=True)):
        print(f"  Label {label}: {count} chunks")
        
    # Display results
    print(evaluation_results.describe())

    # Calculate average scores for numeric columns only
    numeric_columns = evaluation_results.select_dtypes(include=[np.number]).columns
    average_scores = evaluation_results[numeric_columns].mean()
    print("\nAverage Scores:")
    print(average_scores)

    # Plot distributions for numeric columns
    for metric in numeric_columns:
        plot_score_distribution(evaluation_results, metric)

    # Correlation heatmap for numeric columns
    plt.figure(figsize=(12, 10))
    sns.heatmap(evaluation_results[numeric_columns].corr(), annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap of Evaluation Metrics')
    plt.show()

    # Error analysis
    error_analysis(evaluation_results)

    # Display generated answers
    print("\nGenerated Answers:")
    for _, row in evaluation_results.iterrows():
        print(f"Question: {row['question']}")
        print(f"Ground Truth: {row['ground_truth']}")
        print(f"Generated Answer: {row['generated_answer']}")
        print(f"BLEU Score: {row['bleu_score']:.4f}")
        print(f"F1 Score: {row['f1_score']:.4f}")
        print(f"Exact Match: {row['exact_match']}")
        print("--------------------")

    # Clear memory
    rag_system.clear_memory()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
Loading PDFs: 100%|██████████| 30/30 [00:01<00:00, 18.54it/s]


Documents: [Document(page_content="Question: fish appeared in the fossil record during the\n\nAnswer: During the Cambrian Explosion.\n\nContext:\n\nution of fish began about 530 million years ago during the Cambrian explosion. It was during this time that the early chordates developed the skull and the vertebral column, leading to the first craniates and vertebrates. The first fish lineages belong to the Agnatha, or jawless fish. Early examples include Haikouichthys. During the late Cambrian, eel-like jawless fish called the conodonts, and small mostly armoured fish known as ostracoderms, first appeared. Most jawless fish are now extinct; but the extant lampreys may approximate ancient pre-jawed fish. Lampreys belong to the Cyclostomata, which includes the extant hagfish, and this group may have split early on from other agnathans. The first\n\nFull Document (Truncated):\n\nEvolution of fish - Wikipedia .referencetooltip{position:absolute;list-style:none;list-style-image:none;opa li{ba

Splitting Documents: 100%|██████████| 30/30 [00:00<00:00, 323.21it/s]


Text splitting Sankey diagram saved as 'text_splitting_sankey.html'


...

Generated Answer: 1994 design by peter schffer
Retrieved Documents:
Doc 1: Full Document (Truncated):

Coat of arms of South Africa - Wikipedia .referencetooltip{position:absolute;list-style:none;list-style-ima > ge:none;opacity:0;font-size:12px;margin:0;z-index:5;padding:0}...
BLEU Score: 0.0000
ROUGE-L F1: 0.0000
Retrieval Accuracy: 0.0000
BERT Score: -0.1478
Exact Match: 0
F1 Score: 0.0000


Question: when was the first robot used in surgery
Ground Truth: the arthrobot.
Original Context: robot to assist in surgery was the Arthrobot, which was developed and used for the first time in 
Vancouver in 1983.[44] Intimately involved were biomedical engineer, Dr. James McEwen, Geof 
Auchinlec...
=> Retrieving documents...
Retrieved 30 documents.
Using 1 documents for answering.
=> Generating answer...


Evaluating samples: 100%|██████████| 30/30 [08:20<00:00, 16.68s/it]

Generated Answer: 1984 explanation: the context mentions that the arthrobot was used for the first
Retrieved Documents:
Doc 1: Question: when was the first robot used in surgery

Answer: The Arthrobot.

Context:

robot to assist in surgery was the Arthrobot, which was developed and used for the first time in Vancouver in 1983...
BLEU Score: 0.0187
ROUGE-L F1: 0.2667
Retrieval Accuracy: 1.0000
BERT Score: 0.4391
Exact Match: 0
F1 Score: 0.1333
Created visualization for question 0: fish appeared in the fossil record during the...
Created visualization for question 1: when does grey's anatomy season 14 premiere...


Created visualization for question 2: who established the peoples republic of china in 1...
Created visualization for question 3: who holds the most women's wimbledon titles...
Created visualization for question 4: who plays percy in the lost city of z...
Created visualization for question 5: when does life is strange before the storm part 2...
Created visualization for question 6: how many beverly hills cops movies are there...
Created visualization for question 7: where does jinx you owe me a coke come from...
Created visualization for question 8: who is ted talking about in how i met your mother...
Created visualization for question 9: where is the greatest royal rumble taking place...
Created visualization for question 10: the human tendency to mimic other people’s behavio...
Created visualization for question 11: who won season 4 of america's got talent...
Created visualization for question 12: when does model code of conduct come into force...
Created visualization for question 1

Top 10 Worst Predictions:
Question: who sang take that look off your face
Ground Truth: marti webb
Generated Answer: queen queen
BLEU Score: 0.0
F1 Score: 0.0
Exact Match: 0
--------------------
Question: who designed the national coat of arms of south africa
Ground Truth: iaan bekker.
Generated Answer: 1994 design by peter schffer
BLEU Score: 0.0
F1 Score: 0.0
Exact Match: 0
--------------------
Question: when did now thats what i call music come out
Ground Truth: 28 november 1983
Generated Answer: 1993 document 2: title: now thats what i call music
BLEU Score: 0.0
F1 Score: 0.0
Exact Match: 0
--------------------
Question: how many beverly hills cops movies are there
Ground Truth: three.
Generated Answer: 3 document: beverly hills cop (film series) - wikipedia beverly
BLEU Score: 0.0
F1 Score: 0.0
Exact Match: 0
--------------------
Question: where is the greatest royal rumble taking place
Ground Truth: jeddah, saudi arabia
Generated Answer: 1988 wwe event in philadelphia, pennsylvan

goooood good ^

maybe good ^

bad v

In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from rouge_score import rouge_scorer
from REMOVED_SECRET import sentence_bleu, SmoothingFunction
from bert_score import BERTScorer
import os
import re
import fitz  # PyMuPDF

# Import your RAG system
from RAG_UTILS import RAGSystem, EMBEDDING_MODEL_NAME, MODEL_ID, RERANKER_MODEL

#MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"
RERANKER_MODEL = None
NUM_RETRIEVED_DOCS = 5

def clean_text(text):
    # Remove special characters and extra whitespace
    text = re.sub(r'[^\w\s]', '', text)
    return ' '.join(text.split()).lower()

def extract_data_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    
    question_pattern = r"Question: (.*?)\n"
    context_pattern = r"Context:(.*?)(?=Question:|$)"
    answer_pattern = r"Answer: (.*?)(?=\n\n|$)"

    questions = re.findall(question_pattern, text)
    contexts = re.findall(context_pattern, text, re.DOTALL)
    answers = re.findall(answer_pattern, text, re.DOTALL)

    evaluation_data = []
    for q, c, a in zip(questions, contexts, answers):
        evaluation_data.append({
            "question": clean_text(q.strip()),
            "context": c.strip(),
            "ground_truth": clean_text(a.strip())
        })

    return evaluation_data

def calculate_metrics(ground_truth, generated_answer):
    ground_truth = clean_text(ground_truth)
    generated_answer = clean_text(generated_answer)
    
    # BLEU score
    smoothie = SmoothingFunction().method1
    bleu_score = sentence_bleu([ground_truth.split()], generated_answer.split(), 
                               weights=(0.25, 0.25, 0.25, 0.25), 
                               smoothing_function=smoothie)
    
    # Exact match
    exact_match = int(ground_truth == generated_answer)
    
    # F1 score
    ground_truth_set = set(ground_truth.split())
    generated_set = set(generated_answer.split())
    
    if not ground_truth_set and not generated_set:
        f1 = 1.0
    elif not ground_truth_set or not generated_set:
        f1 = 0.0
    else:
        precision = len(ground_truth_set.intersection(generated_set)) / len(generated_set)
        recall = len(ground_truth_set.intersection(generated_set)) / len(ground_truth_set)
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return {
        'bleu_score': bleu_score,
        'exact_match': exact_match,
        'f1_score': f1
    }

def evaluate_rag_system(rag_system, evaluation_data, pdf_folder_path):
    results = []
    
    # Load and process documents
    raw_documents = REMOVED_SECRET(pdf_folder_path)
    processed_documents = REMOVED_SECRET(raw_documents)

    # Build vector database
    knowledge_index = rag_system.build_vector_database(processed_documents)
    
    for sample in tqdm(evaluation_data, desc="Evaluating samples"):
        question = sample['question']
        ground_truth = sample['ground_truth']
        context = sample['context']
        
        # Get RAG system's answer
        answer, relevant_docs, _ = rag_system.answer_with_rag(question, knowledge_index)
        
        # Clean the generated answer
        answer = clean_text(answer)
        
        # Calculate metrics
        metrics = calculate_metrics(ground_truth, answer)
        
        results.append({
            'question': question,
            'ground_truth': ground_truth,
            'generated_answer': answer,
            'bleu_score': metrics['bleu_score'],
            'exact_match': metrics['exact_match'],
            'f1_score': metrics['f1_score'],
        })
    
    return pd.DataFrame(results)

# Main execution
if __name__ == "__main__":
    rag_system = RAGSystem(
        embedding_model_name=EMBEDDING_MODEL_NAME,
        model_id=MODEL_ID,
        reranker_model=RERANKER_MODEL,
    )

    pdf_directory = "nq_samples_good"  # Replace with your PDF directory path
    all_evaluation_data = []
    for filename in os.listdir(pdf_directory):
        if filename.endswith(".pdf"):
            pdf_path = REMOVED_SECRET(pdf_directory, filename)
            all_evaluation_data.extend(extract_data_from_pdf(pdf_path))

    evaluation_results = evaluate_rag_system(rag_system, all_evaluation_data, pdf_directory)

    print("\nEvaluation Results:")
    print(evaluation_results.describe())

    print("\nAverage Scores:")
    print(evaluation_results[['bleu_score', 'exact_match', 'f1_score']].mean())

    # Display some sample results
    print("\nSample Results:")
    for _, row in evaluation_results.sample(n=5).iterrows():
        print(f"Question: {row['question']}")
        print(f"Ground Truth: {row['ground_truth']}")
        print(f"Generated Answer: {row['generated_answer']}")
        print(f"BLEU Score: {row['bleu_score']:.4f}")
        print(f"Exact Match: {row['exact_match']}")
        print(f"F1 Score: {row['f1_score']:.4f}")
        print("--------------------")

    # Save results to CSV
    evaluation_results.to_csv("rag_evaluation_results.csv", index=False)
    print("\nResults saved to rag_evaluation_results.csv")

    rag_system.clear_memory()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
Loading PDFs: 100%|██████████| 30/30 [00:01<00:00, 18.24it/s]


Documents: [Document(page_content='Question: fish appeared in the fossil record during the\n\nAnswer: wiki/Cambrian_explosion" title="Cambrian explosion">Cambrian explosion. It was d\n\nContext:\n\nution of fish began about 530 million years ago during the Cambrian explosion. It was during this time that the early chordates developed the skull and the vertebral column, leading to the first craniates and vertebrates. The first fish lineages belong to the Agnatha, or jawless fish. Early examples include Haikouichthys. During the late Cambrian, eel-like jawless fish called the conodonts, and small mostly armoured fish known as ostracoderms, first appeared. Most jawless fish are now extinct; but the extant lampreys may approximate ancient pre-jawed fish. Lampreys belong to the Cyclostomata, which includes the extant hagfish, and this group may have split early on from other agnathans. The first\n\nFull Document (Truncated):\n\nEvolution of fish - Wikipedia .referencetooltip{position:absolu

Evaluating samples:   0%|          | 0/30 [00:00<?, ?it/s]

=> Retrieving documents...


...

=> Retrieving documents...
Retrieved 30 documents.
Using 1 documents for answering.
=> Generating answer...


Evaluating samples:  97%|█████████▋| 29/30 [31:16<01:04, 64.54s/it]

=> Retrieving documents...
Retrieved 30 documents.
Using 1 documents for answering.
=> Generating answer...


Evaluating samples: 100%|██████████| 30/30 [32:28<00:00, 64.96s/it]



Evaluation Results:
       bleu_score  exact_match   f1_score
count   30.000000         30.0  30.000000
mean     0.003033          0.0   0.161577
std      0.009376          0.0   0.120964
min      0.000000          0.0   0.000000
25%      0.000004          0.0   0.080563
50%      0.000044          0.0   0.133831
75%      0.000281          0.0   0.196875
max      0.043572          0.0   0.480000

Average Scores:
bleu_score     0.003033
exact_match    0.000000
f1_score       0.161577
dtype: float64

Sample Results:
Question: how many goals scored ronaldo in his career
Ground Truth: allers_with_500_or_more_goals titlelist of mens footballers with 500 or more goals670 senior career goals for club and cou context s aveiro goih comm european portuguese kiiiitjinu ioinaidu born 5 february 1985 is a portuguese professional footballer who plays as a forward for italian club juventus and the portugal national team often considered the best player in the world and regarded by many as one of the 